In [2]:
# from two_TrAdaBoostR2 import TwoStageTrAdaBoostR2 ##STrAdaBoost.R2
# from TwoStageTrAdaBoostR2 import TwoStageTrAdaBoostR2 ##two-stage TrAdaBoost.R2

import pandas as pd
import sys
import numpy as np
from pandas import DataFrame
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor

from keras.models import Sequential, load_model, Model
from keras.layers import Input, Dense, Activation, Conv2D, Dropout, Flatten
from keras import optimizers, utils, initializers, regularizers
import keras.backend as K

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler #Importing the StandardScaler

from itertools import combinations

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats.stats import pearsonr
from math import sqrt

#Geo plotting libraries
import geopandas as gdp
from matplotlib.colors import ListedColormap
# import geoplot as glpt

import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn import linear_model
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import KFold
import matplotlib.lines as mlines

import statistics

from scipy.stats import rv_continuous
from scipy.stats import *

from statistics import mean
from sklearn.cluster import KMeans
from scipy.spatial import distance


######### Instance Transfer repositories ####################
from adapt.instance_based import TwoStageTrAdaBoostR2

print("Repositories uploaded!!")

Repositories uploaded!!


In [3]:
from adapt.instance_based import TrAdaBoost, TrAdaBoostR2, TwoStageTrAdaBoostR2
from sklearn.model_selection import GridSearchCV
from adapt.instance_based import KMM

print("Second Upload Completed!!")

Second Upload Completed!!


In [3]:
############################ PoleTelecomm ####################################################################
target_poleTelecomm = ['foo']
colnames_poleTelecomm = ['f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20',
                        'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39',
                        'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'foo']
poleTelecomm_train_df = pd.read_csv('UCI_regression/PoleTelecomm/pol.data', header = None, names = colnames_poleTelecomm)
print("Pole Telecomm Data")
print(poleTelecomm_train_df.shape)

poleTelecomm_test_df = pd.read_csv('UCI_regression/PoleTelecomm/pol.test', header = None, names = colnames_poleTelecomm)
print(poleTelecomm_test_df.shape)

#################### Splitting with small target set and large source and test set #############
poleTelecomm_source_df, poleTelecomm_tgt_df = train_test_split(poleTelecomm_train_df, test_size = 0.05) ## test_size = tgt size

poleTelecomm_tgt_df = poleTelecomm_tgt_df.reset_index(drop = True)
poleTelecomm_source_df = poleTelecomm_source_df.reset_index(drop = True)
print("Target Set: ", poleTelecomm_tgt_df.shape)
print("Source Set: ", poleTelecomm_source_df.shape)
print("Test Set: ", poleTelecomm_test_df.shape)


poleTelecomm_df_test_y = poleTelecomm_test_df[target_poleTelecomm]
poleTelecomm_df_test_X = poleTelecomm_test_df.drop(target_poleTelecomm, axis = 1)

poleTelecomm_df_tgt_y = poleTelecomm_tgt_df[target_poleTelecomm]
poleTelecomm_df_tgt_X = poleTelecomm_tgt_df.drop(target_poleTelecomm, axis = 1)

poleTelecomm_df_source_y = poleTelecomm_source_df[target_poleTelecomm]
poleTelecomm_df_source_X = poleTelecomm_source_df.drop(target_poleTelecomm, axis = 1)

############## Merging the datasets #################
poleTelecomm_X_df = pd.concat([poleTelecomm_df_tgt_X, poleTelecomm_df_source_X], ignore_index=True)
poleTelecomm_y_df = pd.concat([poleTelecomm_df_tgt_y, poleTelecomm_df_source_y], ignore_index=True)

poleTelecomm_np_train_X = poleTelecomm_X_df.to_numpy()
poleTelecomm_np_train_y = poleTelecomm_y_df.to_numpy()

poleTelecomm_np_test_X = poleTelecomm_df_test_X.to_numpy()
poleTelecomm_np_test_y = poleTelecomm_df_test_y.to_numpy()

poleTelecomm_np_train_y_list = poleTelecomm_np_train_y.ravel()
poleTelecomm_np_test_y_list = poleTelecomm_np_test_y.ravel()

src_size_poleTelecomm = len(poleTelecomm_df_source_y)
tgt_size_poleTelecomm = len(poleTelecomm_df_tgt_y)

print("---------------------------")

Pole Telecomm Data
(5000, 49)
(10000, 49)
Target Set:  (250, 49)
Source Set:  (4750, 49)
Test Set:  (10000, 49)
---------------------------


In [52]:
######################################## Phase 1: Seeding Technique ###################################################
kmeans = KMeans(n_clusters = 9, random_state=0).fit(poleTelecomm_df_tgt)

poleTelecomm_alternate_df = poleTelecomm_tgt_df.copy()
poleTelecomm_alternate_df_np = poleTelecomm_tgt_df.to_numpy()

idxlist = []
poleTelecomm_new_df_list = []

for rowkm in kmeans.cluster_centers_:
    mindist = -99
    rowidx = 0
    idx = 0
    for row in poleTelecomm_alternate_df_np:
        dst = distance.euclidean(row, rowkm)

        if(dst >= mindist):
            mindist = dst
            rowidx = idx
            rowval = row

        idx = idx + 1

    print("Row selected: ", rowidx) #Alternate_df.loc[rowidx,:]\
    print("Min. distance: ", mindist)
    print("Matrix shape: ", poleTelecomm_alternate_df_np.shape)
    poleTelecomm_new_df_list.append(rowval)
    poleTelecomm_alternate_df = np.delete(poleTelecomm_alternate_df_np, rowidx, 0)
    idxlist.append(rowidx)


poleTelecomm_new_df = pd.DataFrame(np.vstack(poleTelecomm_new_df_list))
poleTelecomm_new_df.shape

Row selected:  224
Min. distance:  176.10275143608882
Matrix shape:  (250, 49)
Row selected:  224
Min. distance:  156.17399129034115
Matrix shape:  (250, 49)
Row selected:  162
Min. distance:  181.4962869736561
Matrix shape:  (250, 49)
Row selected:  121
Min. distance:  155.54445320575377
Matrix shape:  (250, 49)
Row selected:  224
Min. distance:  178.48785653527122
Matrix shape:  (250, 49)
Row selected:  224
Min. distance:  190.61349218080167
Matrix shape:  (250, 49)
Row selected:  162
Min. distance:  181.3077601931313
Matrix shape:  (250, 49)
Row selected:  121
Min. distance:  194.5782573072728
Matrix shape:  (250, 49)
Row selected:  121
Min. distance:  173.1592752796818
Matrix shape:  (250, 49)


(9, 49)

In [53]:
##################################################### Phase 2: Seeding ################################################

poleTelecomm_alternate_source_df = poleTelecomm_source_df.copy()
poleTelecomm_alternate_source_df_np = poleTelecomm_alternate_source_df.to_numpy()

idxlist2 = []
poleTelecomm_final_df_list = []

for row_nm in poleTelecomm_new_df_list:
    min_dist = -99
    row_idx = 0
    idx_val = 0
    for row_alt in poleTelecomm_alternate_df_np:
        dst = distance.euclidean(row_alt, row_nm)
        if(dst >= mindist):
            min_dist = dst
            row_idx = idx_val
            row_val = row_alt

        idx_val = idx_val + 1

    print("Row selected: ", row_idx) #Alternate_df.loc[rowidx,:]\
    print("Min. distance: ", min_dist)
    print("Matrix shape: ", poleTelecomm_alternate_source_df_np.shape)
    poleTelecomm_final_df_list.append(row_val)
    poleTelecomm_alternate_source_df_np = np.delete(poleTelecomm_alternate_source_df_np, row_idx, 0)
    idxlist2.append(row_idx)


poleTelecomm_final_df = pd.DataFrame(np.vstack(poleTelecomm_final_df_list), columns= poleTelecomm_alternate_source_df.columns)

print("----------------------------------------------")
print("Shape of source before :",poleTelecomm_source_df.shape)
poleTelecomm_source_df = pd.DataFrame(np.vstack(poleTelecomm_alternate_source_df_np), columns= poleTelecomm_source_df.columns)
print("Shape of source after :", poleTelecomm_source_df.shape)

print("----------------------------------------------")
print("Shape of target before :", poleTelecomm_tgt_df.shape)
poleTelecomm_tgt_df = pd.concat([poleTelecomm_tgt_df, poleTelecomm_final_df], ignore_index=True)
print("Shape of target after :", poleTelecomm_tgt_df.shape)

print("----------------------------------------------")

Row selected:  248
Min. distance:  174.4448336867561
Matrix shape:  (4750, 49)
Row selected:  248
Min. distance:  174.4448336867561
Matrix shape:  (4749, 49)
Row selected:  249
Min. distance:  177.6400855662933
Matrix shape:  (4748, 49)
Row selected:  248
Min. distance:  182.23062311258226
Matrix shape:  (4747, 49)
Row selected:  248
Min. distance:  174.4448336867561
Matrix shape:  (4746, 49)
Row selected:  248
Min. distance:  174.4448336867561
Matrix shape:  (4745, 49)
Row selected:  249
Min. distance:  177.6400855662933
Matrix shape:  (4744, 49)
Row selected:  248
Min. distance:  182.23062311258226
Matrix shape:  (4743, 49)
Row selected:  248
Min. distance:  182.23062311258226
Matrix shape:  (4742, 49)
----------------------------------------------
Shape of source before : (4750, 49)
Shape of source after : (4741, 49)
----------------------------------------------
Shape of target before : (250, 49)
Shape of target after : (259, 49)
----------------------------------------------


In [54]:
poleTelecomm_source_df.columns

Index(['f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11',
       'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21',
       'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31',
       'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f41',
       'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'foo'],
      dtype='object')

In [55]:
################################## Finding best instances from the source dataset ######################################################
poleTelecomm_source_df["ManDis"] = ""

poleTelecomm_tgt_df_mean = []
prow = poleTelecomm_tgt_df.mean()
poleTelecomm_tgt_df_mean = [prow.f1, prow.f2, prow.f3, prow.f4, prow.f5, prow.f6, prow.f7, prow.f8, prow.f9, 
                            prow.f10, prow.f11, prow.f12, prow.f13, prow.f14, prow.f15, prow.f16, prow.f17, 
                            prow.f18, prow.f19, prow.f20, prow.f21, prow.f22, prow.f23, prow.f24, prow.f25, 
                            prow.f26, prow.f27, prow.f28, prow.f29, prow.f30, prow.f31, prow.f32, prow.f33, 
                            prow.f34, prow.f35, prow.f36, prow.f37, prow.f38, prow.f39, prow.f40, prow.f41,
                            prow.f42, prow.f43, prow.f44, prow.f45, prow.f46, prow.f47, prow.f48, prow.foo]

rowidx = 0

for row in poleTelecomm_source_df.itertuples():
    row_list =[row.f1, row.f2, row.f3, row.f4, row.f5, row.f6, row.f7, row.f8, row.f9, 
            row.f10, row.f11, row.f12, row.f13, row.f14, row.f15, row.f16, row.f17, 
            row.f18, row.f19, row.f20, row.f21, row.f22, row.f23, row.f24, row.f25, 
            row.f26, row.f27, row.f28, row.f29, row.f30, row.f31, row.f32, row.f33, 
            row.f34, row.f35, row.f36, row.f37, row.f38, row.f39, row.f40, row.f41,
            row.f42, row.f43, row.f44, row.f45, row.f46, row.f47, row.f48, row.foo]

    man_dis = 0
    for i in range(0, len(row_list)):
        tempval = poleTelecomm_tgt_df_mean[i] - row_list[i]
        man_dis = man_dis + abs(tempval)

    poleTelecomm_source_df.loc[rowidx,"ManDis"] = man_dis
    rowidx = rowidx + 1

poleTelecomm_source_df = poleTelecomm_source_df.sort_values(by =['ManDis'])
poleTelecomm_source_df = poleTelecomm_source_df.head(1500) 
poleTelecomm_source_df = poleTelecomm_source_df.drop(['ManDis'], axis =1)
poleTelecomm_source_df = poleTelecomm_source_df.reset_index(drop=True)

#################### Splitting with small target set and large source and test set #############
print("Target Set: ", poleTelecomm_tgt_df.shape)
print("Source Set: ", poleTelecomm_source_df.shape)
print("Test Set: ", poleTelecomm_test_df.shape)


poleTelecomm_df_test_y = poleTelecomm_test_df[target_poleTelecomm]
poleTelecomm_df_test_X = poleTelecomm_test_df.drop(target_poleTelecomm, axis = 1)

poleTelecomm_df_tgt_y = poleTelecomm_tgt_df[target_poleTelecomm]
poleTelecomm_df_tgt_X = poleTelecomm_tgt_df.drop(target_poleTelecomm, axis = 1)

poleTelecomm_df_source_y = poleTelecomm_source_df[target_poleTelecomm]
poleTelecomm_df_source_X = poleTelecomm_source_df.drop(target_poleTelecomm, axis = 1)

############## Merging the datasets #################
poleTelecomm_X_df = pd.concat([poleTelecomm_df_tgt_X, poleTelecomm_df_source_X], ignore_index=True)
poleTelecomm_y_df = pd.concat([poleTelecomm_df_tgt_y, poleTelecomm_df_source_y], ignore_index=True)

poleTelecomm_np_train_X = poleTelecomm_X_df.to_numpy()
poleTelecomm_np_train_y = poleTelecomm_y_df.to_numpy()

poleTelecomm_np_test_X = poleTelecomm_df_test_X.to_numpy()
poleTelecomm_np_test_y = poleTelecomm_df_test_y.to_numpy()

poleTelecomm_np_train_y_list = poleTelecomm_np_train_y.ravel()
poleTelecomm_np_test_y_list = poleTelecomm_np_test_y.ravel()

src_size_poleTelecomm = len(poleTelecomm_df_source_y)
tgt_size_poleTelecomm = len(poleTelecomm_df_tgt_y)

print("---------------------------")

Target Set:  (259, 49)
Source Set:  (1500, 49)
Test Set:  (10000, 49)
---------------------------


In [49]:
################################## Transfer Learning specifications #########################################################
src_idx_poleTelecomm = np.arange(start=0, stop=(src_size_poleTelecomm - 1), step=1)
tgt_idx_poleTelecomm = np.arange(start=src_size_poleTelecomm, stop=((src_size_poleTelecomm + tgt_size_poleTelecomm)-1), step=1)

def get_estimator(**kwargs):
    return DecisionTreeRegressor(max_depth = 6)

kwargs_TwoTrAda = {'steps': 30,
                    'fold': 10,
                  'learning_rate': 0.1}

In [4]:
########################### TwoStagetrAdaBoostR2 PoleTelecomm #######################################

from adapt.instance_based import TrAdaBoost, TrAdaBoostR2, TwoStageTrAdaBoostR2

print("Two-Stage TrAdaboost.R2")
print("-------------------------------------------")

r2scorelist_TwoTrAda_poleTelecomm = []
rmselist_TwoTrAda_poleTelecomm = []

for x in range(0, 10):

    model_TwoTrAda_poleTelecomm = TwoStageTrAdaBoostR2(get_estimator = get_estimator, n_estimators = 1000, cv=10) #, kwargs_TwoTrAda)
    model_TwoTrAda_poleTelecomm.fit(poleTelecomm_np_train_X, poleTelecomm_np_train_y_list, src_idx_poleTelecomm, tgt_idx_poleTelecomm)

    y_pred_TwoTrAda_poleTelecomm = model_TwoTrAda_poleTelecomm.predict(poleTelecomm_np_test_X)

    mse_TwoTrAda_poleTelecomm = sqrt(mean_squared_error(poleTelecomm_np_test_y, y_pred_TwoTrAda_poleTelecomm))
    rmselist_TwoTrAda_poleTelecomm.append(mse_TwoTrAda_poleTelecomm)
        
    r2_score_TwoTrAda_poleTelecomm = pearsonr(poleTelecomm_np_test_y_list, y_pred_TwoTrAda_poleTelecomm)
    r2_score_TwoTrAda_poleTelecomm = (r2_score_TwoTrAda_poleTelecomm[0])**2
    r2scorelist_TwoTrAda_poleTelecomm.append(r2_score_TwoTrAda_poleTelecomm)

print("RMSE of Two-Stage TrAdaboost.R2:", statistics.mean(rmselist_TwoTrAda_poleTelecomm))
print("R^2 of TrAdaboostR2:", statistics.mean(r2scorelist_TwoTrAda_poleTelecomm))
print("\n")
print("RMSE of Two-Stage TrAdaboost.R2:", rmselist_TwoTrAda_poleTelecomm)
print("R^2 of TrAdaboostR2:", r2scorelist_TwoTrAda_poleTelecomm)


print("-------------------------------------------")

Two-Stage TrAdaboost.R2
-------------------------------------------


NameError: name 'get_estimator' is not defined

In [7]:
########################### AdaBoostR2 Transfer Learning PoleTelecomm #####################################################

from sklearn.ensemble import AdaBoostRegressor


print("Adaboost.R2 Transfer Learning")
print("-------------------------------------------")

r2scorelist_AdaTL_poleTelecomm = []
rmselist_AdaTL_poleTelecomm = []

for x in range(0, 10):

    model_AdaTL_poleTelecomm = AdaBoostRegressor(DecisionTreeRegressor(max_depth = 8), learning_rate=0.01, n_estimators=500) 
    model_AdaTL_poleTelecomm.fit(poleTelecomm_np_train_X, poleTelecomm_np_train_y_list)

    y_pred_AdaTL_poleTelecomm = model_AdaTL_poleTelecomm.predict(poleTelecomm_np_test_X)

    mse_AdaTL_poleTelecomm = sqrt(mean_squared_error(poleTelecomm_np_test_y, y_pred_AdaTL_poleTelecomm))
    rmselist_AdaTL_poleTelecomm.append(mse_AdaTL_poleTelecomm)
        
    r2_score_AdaTL_poleTelecomm = pearsonr(poleTelecomm_np_test_y_list, y_pred_AdaTL_poleTelecomm)
    r2_score_AdaTL_poleTelecomm = (r2_score_AdaTL_poleTelecomm[0])**2
    r2scorelist_AdaTL_poleTelecomm.append(r2_score_AdaTL_poleTelecomm)

print("RMSE of Adaboost.R2:", statistics.mean(rmselist_AdaTL_poleTelecomm))
print("R^2 of TrAdaboostR2:", statistics.mean(r2scorelist_AdaTL_poleTelecomm))
print("\n")
print("RMSE of Adaboost.R2:", rmselist_AdaTL_poleTelecomm)
print("R^2 of TrAdaboostR2:", r2scorelist_AdaTL_poleTelecomm)


print("-------------------------------------------")

Adaboost.R2 Transfer Learning
-------------------------------------------
RMSE of Adaboost.R2: 7.218640787985613
R^2 of TrAdaboostR2: 0.9709765279509841
-------------------------------------------


In [9]:
########################### AdaBoostR2 PoleTelecomm #####################################################

from sklearn.ensemble import AdaBoostRegressor


print("Adaboost.R2")
print("-------------------------------------------")

r2scorelist_Ada_poleTelecomm = []
rmselist_Ada_poleTelecomm = []

for x in range(0, 10):

    model_Ada_poleTelecomm = AdaBoostRegressor(DecisionTreeRegressor(max_depth = 8), learning_rate=0.01, n_estimators=500) 
    model_Ada_poleTelecomm.fit(poleTelecomm_np_train_X, poleTelecomm_np_train_y_list)

    y_pred_Ada_poleTelecomm = model_Ada_poleTelecomm.predict(poleTelecomm_np_test_X)

    mse_Ada_poleTelecomm = sqrt(mean_squared_error(poleTelecomm_np_test_y, y_pred_Ada_poleTelecomm))
    rmselist_Ada_poleTelecomm.append(rmselist_Ada_poleTelecomm)
        
    r2_score_Ada_poleTelecomm = pearsonr(poleTelecomm_np_test_y_list, y_pred_Ada_poleTelecomm)
    r2_score_Ada_poleTelecomm = (r2_score_Ada_poleTelecomm[0])**2
    r2scorelist_Ada_poleTelecomm.append(r2scorelist_Ada_poleTelecomm)


print("RMSE of Adaboost.R2:", statistics.mean(rmselist_Ada_poleTelecomm))
print("R^2 of TrAdaboostR2:", statistics.mean(r2scorelist_Ada_poleTelecomm))
print("\n")
print("RMSE of Adaboost.R2:", rmselist_Ada_poleTelecomm)
print("R^2 of TrAdaboostR2:", r2scorelist_Ada_poleTelecomm)


print("-------------------------------------------")

Adaboost.R2
-------------------------------------------
RMSE of Adaboost.R2: 7.260392122741309
R^2 of TrAdaboostR2: 0.9707171716119096
-------------------------------------------


In [ ]:
######################### Kernel Mean Matching PoleTelecomm #######################################

from adapt.instance_based import KMM


print("Kernel Mean Matching")
print("-------------------------------------------")

r2scorelist_KMM_poleTelecomm = []
rmselist_KMM_poleTelecomm = []

for x in range(0, 10):

    model_KMM_poleTelecomm = KMM(get_estimator = get_estimator)
    model_KMM_poleTelecomm.fit(poleTelecomm_np_train_X, poleTelecomm_np_train_y_list, src_idx_poleTelecomm, tgt_idx_poleTelecomm)

    y_pred_KMM_poleTelecomm = model_KMM_elevators.predict(poleTelecomm_df_test_X) ##Using dataframe instead of the numpy matrix

    mse_KMM_poleTelecomm = sqrt(mean_squared_error(poleTelecomm_np_test_y, y_pred_KMM_poleTelecomm))
    rmselist_KMM_poleTelecomm.append(rmselist_KMM_poleTelecomm)
        
    r2_score_KMM_poleTelecomm = pearsonr(poleTelecomm_np_test_y_list, y_pred_KMM_poleTelecomm)
    r2_score_KMM_poleTelecomm = (r2_score_KMM_poleTelecomm[0])**2
    r2scorelist_KMM_poleTelecomm.append(r2scorelist_KMM_poleTelecomm)


print("RMSE of KMM:", statistics.mean(rmselist_KMM_poleTelecomm))
print("R^2 of TrAdaboostR2:", statistics.mean(r2scorelist_KMM_poleTelecomm))
print("\n")
print("RMSE of KMM:", rmselist_KMM_poleTelecomm)
print("R^2 of TrAdaboostR2:", r2scorelist_KMM_poleTelecomm)


#model_KMM_concrete.weights_   #To get the weights of the algorithm

print("-------------------------------------------")

In [10]:
######################### Gradient Boosting Regression Transfer Learning PoleTelecomm #######################################

from sklearn.ensemble import GradientBoostingRegressor


print("Gradient Boosting Regression Transfer Learning")
print("-------------------------------------------")

r2scorelist_GBRTL_poleTelecomm = []
rmselist_GBRTL_poleTelecomm = []

for x in range(0, 10):

    model_GBRTL_poleTelecomm = GradientBoostingRegressor(learning_rate=0.01, max_depth=4, n_estimators=1000, subsample=0.5)
    model_GBRTL_poleTelecomm.fit(poleTelecomm_np_train_X, poleTelecomm_np_train_y_list)

    y_pred_GBRTL_poleTelecomm = model_GBRTL_poleTelecomm.predict(poleTelecomm_df_test_X) ##Using dataframe instead of the numpy matrix

    mse_GBRTL_poleTelecomm = sqrt(mean_squared_error(poleTelecomm_np_test_y, y_pred_GBRTL_poleTelecomm))
    rmselist_GBRTL_poleTelecomm.append(rmselist_GBRTL_poleTelecomm)
        
    r2_score_GBRTL_poleTelecomm = pearsonr(poleTelecomm_np_test_y_list, y_pred_GBRTL_poleTelecomm)
    r2_score_GBRTL_poleTelecomm = (r2_score_GBRTL_poleTelecomm[0])**2
    r2scorelist_GBRTL_poleTelecomm.append(r2scorelist_GBRTL_poleTelecomm)


print("RMSE of KMM:", statistics.mean(rmselist_GBRTL_poleTelecomm))
print("R^2 of TrAdaboostR2:", statistics.mean(r2scorelist_GBRTL_poleTelecomm))
print("\n")
print("RMSE of KMM:", rmselist_GBRTL_poleTelecomm)
print("R^2 of TrAdaboostR2:", r2scorelist_GBRTL_poleTelecomm)


print("-------------------------------------------")

Gradient Boosting Regression Transfer Learning
-------------------------------------------
RMSE of KMM: 7.914473672205558
R^2 of TrAdaboostR2: 0.9642087093073816
-------------------------------------------


In [11]:
######################### Gradient Boosting Regression PoleTelecomm #######################################

from sklearn.ensemble import GradientBoostingRegressor


print("Gradient Boosting Regression")
print("-------------------------------------------")

r2scorelist_GBR_poleTelecomm = []
rmselist_GBR_poleTelecomm = []

for x in range(0, 10):

    model_GBR_poleTelecomm = GradientBoostingRegressor(learning_rate=0.01, max_depth=4, n_estimators=1000, subsample=0.5)
    model_GBR_poleTelecomm.fit(poleTelecomm_df_tgt_X, poleTelecomm_df_tgt_y)

    y_pred_GBR_poleTelecomm = model_GBR_poleTelecomm.predict(poleTelecomm_df_test_X) ##Using dataframe instead of the numpy matrix

    mse_GBR_poleTelecomm = sqrt(mean_squared_error(poleTelecomm_np_test_y, y_pred_GBR_poleTelecomm))
    rmselist_GBR_poleTelecomm.append(rmselist_GBR_poleTelecomm)
        
    r2_score_GBR_poleTelecomm = pearsonr(poleTelecomm_np_test_y_list, y_pred_GBR_poleTelecomm)
    r2_score_GBR_poleTelecomm = (r2_score_GBR_poleTelecomm[0])**2
    r2scorelist_GBR_poleTelecomm.append(r2scorelist_GBR_poleTelecomm)

print("RMSE of KMM:", statistics.mean(rmselist_GBR_poleTelecomm))
print("R^2 of TrAdaboostR2:", statistics.mean(r2scorelist_GBR_poleTelecomm))
print("\n")
print("RMSE of KMM:", rmselist_GBR_poleTelecomm)
print("R^2 of TrAdaboostR2:", r2scorelist_GBR_poleTelecomm)


print("-------------------------------------------")

Gradient Boosting Regression
-------------------------------------------


/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


RMSE of KMM: 13.827574736293434
R^2 of TrAdaboostR2: 0.889892496801516
-------------------------------------------


In [ ]:
################ Transformed TrAdaBoost.R2 PoleTelecomm #############################################

from adapt.instance_based import TrAdaBoost, TrAdaBoostR2, TwoStageTrAdaBoostR2


weights_KMM = model_KMM_poleTelecomm.weights_

val_mean = np.mean(weights_KMM)
weights_KMM = np.append(weights_KMM, val_mean)

poleTelecomm_source_df_X_trans = poleTelecomm_source_df_X.apply(lambda poleTelecomm_source_df_X: poleTelecomm_source_df_X * weights_KMM)
#concrete_source_df_X.multiply(weights_KMM, axis=0)

# concrete_np_train_X_trans = concrete_source_df_X_trans.to_numpy()

poleTelecomm_X_df = pd.concat([poleTelecomm_tgt_df_X, poleTelecomm_source_df_X_trans], ignore_index=True)
poleTelecomm_y_df = pd.concat([poleTelecomm_tgt_df_y, poleTelecomm_source_df_y], ignore_index=True)

poleTelecomm_np_train_X = poleTelecomm_X_df.to_numpy()
poleTelecomm_np_train_y = poleTelecomm_y_df.to_numpy()

poleTelecomm_np_test_X = poleTelecomm_test_df_X.to_numpy()
poleTelecomm_np_test_y = poleTelecomm_test_df_y.to_numpy()

poleTelecomm_np_train_y_list = poleTelecomm_np_train_y.ravel()
poleTelecomm_np_test_y_list = poleTelecomm_np_test_y.ravel()

# print(concrete_np_train_y_list.shape, concrete_np_train_X_trans.shape)

##################################################################
print("Transformed Two-Stage TrAdaboost.R2")
print("-------------------------------------------")

r2scorelist_stradaboost_poleTelecomm = []
rmselist_stradaboost_poleTelecomm = []

for x in range(0, 10):

    model_TwoTrAda_poleTelecomm = TwoStageTrAdaBoostR2(get_estimator = get_estimator, n_estimators = 100, cv=10) #, kwargs_TwoTrAda)
    model_TwoTrAda_poleTelecomm.fit(poleTelecomm_np_train_X, poleTelecomm_np_train_y_list, src_idx_poleTelecomm, tgt_idx_poleTelecomm)

    y_pred_TwoTrAda_poleTelecomm = model_TwoTrAda_poleTelecomm.predict(poleTelecomm_np_test_X)

    mse_TwoTrAda_poleTelecomm = sqrt(mean_squared_error(poleTelecomm_np_test_y, y_pred_TwoTrAda_poleTelecomm))
    rmselist_TwoTrAda_poleTelecomm.append(rmselist_stradaboost_poleTelecomm)
        
    r2_score_TwoTrAda_poleTelecomm = pearsonr(poleTelecomm_np_test_y_list, y_pred_TwoTrAda_poleTelecomm)
    r2_score_TwoTrAda_poleTelecomm = (r2_score_TwoTrAda_poleTelecomm[0])**2
    r2scorelist_TwoTrAda_poleTelecomm.append(r2scorelist_stradaboost_poleTelecomm)


print("RMSE of Two-Stage TrAdaboost.R2:", statistics.mean(rmselist_stradaboost_poleTelecomm))
print("R^2 of TrAdaboostR2:", statistics.mean(r2scorelist_stradaboost_poleTelecomm))
print("\n")
print("RMSE of Two-Stage TrAdaboost.R2:", rmselist_stradaboost_poleTelecomm)
print("R^2 of TrAdaboostR2:", r2scorelist_stradaboost_poleTelecomm)


print("-------------------------------------------")

In [12]:
#################################### STrAdaBoost.R2 PoleTelecomm ################################################################
from two_TrAdaBoostR2 import TwoStageTrAdaBoostR2

sample_size = [len(poleTelecomm_df_tgt_X), len(poleTelecomm_df_source_X)]
n_estimators = 100
steps = 30
fold = 10
random_state = np.random.RandomState(1)

r2scorelist_stradaboost_poleTelecomm = []
rmselist_stradaboost_poleTelecomm = []

for x in range(0, 10):

    model_stradaboost_poleTelecomm = TwoStageTrAdaBoostR2(DecisionTreeRegressor(max_depth=6),
                        n_estimators = n_estimators, sample_size = sample_size,
                        steps = steps, fold = fold, random_state = random_state)


    model_stradaboost_poleTelecomm.fit(poleTelecomm_np_train_X, poleTelecomm_np_train_y_list)
    y_pred_stradaboost_poleTelecomm = model_stradaboost_poleTelecomm.predict(poleTelecomm_np_test_X)


    mse_stradaboost_poleTelecomm = sqrt(mean_squared_error(poleTelecomm_np_test_y, y_pred_stradaboost_poleTelecomm))
    rmselist_stradaboost_poleTelecomm.append(rmselist_stradaboost_poleTelecomm)
        
    r2_score_stradaboost_poleTelecomm = pearsonr(poleTelecomm_np_test_y_list, y_pred_stradaboost_poleTelecomm)
    r2_score_stradaboost_poleTelecomm = (r2_score_stradaboost_poleTelecomm[0])**2
    r2scorelist_stradaboost_poleTelecomm.append(r2scorelist_stradaboost_poleTelecomm)


print("RMSE of STrAdaboostR2:", statistics.mean(rmselist_stradaboost_poleTelecomm))
print("R^2 of STrAdaboostR2:", statistics.mean(r2_score_stradaboost_poleTelecomm))
print("\n")
print("RMSE of STrAdaboostR2:", rmselist_stradaboost_poleTelecomm)
print("R^2 of STrAdaboostR2:", r2_score_stradaboost_poleTelecomm)

Inside STrAdaBoost.R2


KeyboardInterrupt: 

In [4]:
################################### Abalone ###########################################
#### range: [0.0 - 1.130]
#### Mid of correlation variable: Whole_weight
#### [0, 0.12] [0.12, 0.15], [0.15, 1.130]
#### Target variable: Rings
#######################################################################################
target_var_abalone = ['Rings']
colnames_abalone = ['Sex', 'Length', 'Diameter', 'Height', 'Whole_weight', 'Shucked_weight', 'Viscera_weight', 'Shell_weight', 'Rings']
AbaloneData_df = pd.read_csv('UCI_regression/Abalone/abalone.data', header = None, names = colnames_abalone)

gender = {'M': 1,'F': 2, 'I': 3} 
AbaloneData_df.Sex = [gender[item] for item in AbaloneData_df.Sex] 

print("Abalone Data")
print(AbaloneData_df.shape)

########## Corr Abalone ################
# print("The correlation matrix is: ")
# print(AbaloneData_df.corr()['Rings'])

##################### Splitting in 3 equal parts #######################################
# print(AbaloneData_df.sort_values('Whole_weight')['Whole_weight'])
# print(len(AbaloneData_df.loc[(AbaloneData_df['Whole_weight'] <= 0.5)]))
# print(len(AbaloneData_df.loc[(AbaloneData_df['Whole_weight'] > 0.5) & (AbaloneData_df['Whole_weight'] <= 1.0)]))
# print(len(AbaloneData_df.loc[(AbaloneData_df['Whole_weight'] > 1.0)]))

#################### Splitting with small target set and large source and test set #############
# AbaloneData_df_remain, AbaloneData_df_tgt = train_test_split(AbaloneData_df, test_size = 0.05) ## test_size = tgt size
# AbaloneData_df_source, AbaloneData_df_test = train_test_split(AbaloneData_df_remain, test_size = 0.3) ## test_size = tgt size
# print(AbaloneData_df_tgt.shape, AbaloneData_df_source.shape, AbaloneData_df_test.shape)


drop_col_abalone = ['Whole_weight']

abalone_tgt_df = AbaloneData_df.loc[(AbaloneData_df['Whole_weight'] <= 0.5)]
abalone_tgt_df = abalone_tgt_df.drop(drop_col_abalone, axis = 1)
abalone_tgt_df = abalone_tgt_df.reset_index(drop=True)
print("Target Set: ",abalone_tgt_df.shape)

abalone_source_df = AbaloneData_df.loc[(AbaloneData_df['Whole_weight'] > 0.5) & (AbaloneData_df['Whole_weight'] <= 1.0)]
abalone_source_df = abalone_source_df.drop(drop_col_abalone, axis = 1)
abalone_source_df = abalone_source_df.reset_index(drop=True)
print("Source Set: ",abalone_source_df.shape)

abalone_test_df = AbaloneData_df.loc[(AbaloneData_df['Whole_weight'] > 1.0)]
abalone_test_df = abalone_test_df.drop(drop_col_abalone, axis = 1)
abalone_test_df = abalone_test_df.reset_index(drop=True)
print("Test Set: ",abalone_test_df.shape)

#################### Splitting into features and target ####################
target_column_abalone = ['Rings']

abalone_tgt_df_y = abalone_tgt_df[target_column_abalone]
abalone_tgt_df_X = abalone_tgt_df.drop(target_column_abalone, axis = 1)

abalone_source_df_y = abalone_source_df[target_column_abalone]
abalone_source_df_X = abalone_source_df.drop(target_column_abalone, axis = 1)

abalone_test_df_y = abalone_test_df[target_column_abalone]
abalone_test_df_X = abalone_test_df.drop(target_column_abalone, axis = 1)

############## Merging the datasets ##########################################
abalone_X_df = pd.concat([abalone_tgt_df_X, abalone_source_df_X], ignore_index=True)
abalone_y_df = pd.concat([abalone_tgt_df_y, abalone_source_df_y], ignore_index=True)

abalone_np_train_X = abalone_X_df.to_numpy()
abalone_np_train_y = abalone_y_df.to_numpy()

abalone_np_test_X = abalone_test_df_X.to_numpy()
abalone_np_test_y = abalone_test_df_y.to_numpy()

abalone_np_train_y_list = abalone_np_train_y.ravel()
abalone_np_test_y_list = abalone_np_test_y.ravel()

src_size_abalone = len(abalone_source_df_y)
tgt_size_abalone = len(abalone_tgt_df_y)

print("---------------------------")

###############################################################################################################################################

Abalone Data
(4177, 9)
Target Set:  (1241, 8)
Source Set:  (1465, 8)
Test Set:  (1471, 8)
---------------------------


In [9]:
######################################## Phase 1: Seeding Technique (Abalone) ###################################################
kmeans = KMeans(n_clusters = 120, random_state=0).fit(abalone_tgt_df)

abalone_alternate_df = abalone_tgt_df.copy()
abalone_alternate_df_np = abalone_tgt_df.to_numpy()

idxlist = []
abalone_new_df_list = []

for rowkm in kmeans.cluster_centers_:
    mindist = -99
    rowidx = 0
    idx = 0
    for row in abalone_alternate_df_np:
        dst = distance.euclidean(row, rowkm)

        if(dst >= mindist):
            mindist = dst
            rowidx = idx
            rowval = row

        idx = idx + 1

#     print("Row selected: ", rowidx) #Alternate_df.loc[rowidx,:]\
#     print("Min. distance: ", mindist)
#     print("Matrix shape: ", abalone_alternate_df_np.shape)
    abalone_new_df_list.append(rowval)
    abalone_alternate_df = np.delete(abalone_alternate_df_np, rowidx, 0)
    idxlist.append(rowidx)


abalone_new_df = pd.DataFrame(np.vstack(abalone_new_df_list))

print("The shape of the extracted data is: ")
print(abalone_new_df.shape)
print("-------------------------------------------------------------------")

The shape of the extracted data is: 
(120, 8)
-------------------------------------------------------------------


In [10]:
##################################################### Phase 2: Seeding (Abalone) ################################################

abalone_alternate_source_df = abalone_source_df.copy()
abalone_alternate_source_df_np = abalone_alternate_source_df.to_numpy()

idxlist2 = []
abalone_final_df_list = []

for row_nm in abalone_new_df_list:
    min_dist = -99
    row_idx = 0
    idx_val = 0
    for row_alt in abalone_alternate_source_df_np:
        dst = distance.euclidean(row_alt, row_nm)
        if(dst >= mindist):
            min_dist = dst
            row_idx = idx_val
            row_val = row_alt

        idx_val = idx_val + 1

#     print("Row selected: ", row_idx) #Alternate_df.loc[rowidx,:]\
#     print("Min. distance: ", min_dist)
#     print("Matrix shape: ", abalone_alternate_source_df_np.shape)
    abalone_final_df_list.append(row_val)
    abalone_alternate_source_df_np = np.delete(abalone_alternate_source_df_np, row_idx, 0)
    idxlist2.append(row_idx)


abalone_final_df = pd.DataFrame(np.vstack(abalone_final_df_list), columns= abalone_alternate_source_df.columns)

print("----------------------------------------------")
print("Shape of source before :",abalone_source_df.shape)
abalone_source_df = pd.DataFrame(np.vstack(abalone_alternate_source_df_np), columns= abalone_source_df.columns)
print("Shape of source after :", abalone_source_df.shape)

print("----------------------------------------------")
print("Shape of target before :", abalone_tgt_df.shape)
abalone_tgt_df = pd.concat([abalone_tgt_df, abalone_final_df], ignore_index=True)
print("Shape of target after :", abalone_tgt_df.shape)

print("----------------------------------------------")

----------------------------------------------
Shape of source before : (1465, 8)
Shape of source after : (1345, 8)
----------------------------------------------
Shape of target before : (1241, 8)
Shape of target after : (1361, 8)
----------------------------------------------


In [16]:
#################### This is when only applying variance filtering ###############################################
target_column_abalone = ['Rings']

abalone_tgt_df_y = abalone_tgt_df[target_column_abalone]
abalone_tgt_df_X = abalone_tgt_df.drop(target_column_abalone, axis = 1)

abalone_source_df_y = abalone_source_df[target_column_abalone]
abalone_source_df_X = abalone_source_df.drop(target_column_abalone, axis = 1)

abalone_test_df_y = abalone_test_df[target_column_abalone]
abalone_test_df_X = abalone_test_df.drop(target_column_abalone, axis = 1)

############## Merging the datasets ##########################################
abalone_X_df = pd.concat([abalone_tgt_df_X, abalone_source_df_X], ignore_index=True)
abalone_y_df = pd.concat([abalone_tgt_df_y, abalone_source_df_y], ignore_index=True)

abalone_np_train_X = abalone_X_df.to_numpy()
abalone_np_train_y = abalone_y_df.to_numpy()

abalone_np_test_X = abalone_test_df_X.to_numpy()
abalone_np_test_y = abalone_test_df_y.to_numpy()

abalone_np_train_y_list = abalone_np_train_y.ravel()
abalone_np_test_y_list = abalone_np_test_y.ravel()

src_size_abalone = len(abalone_source_df_y)
tgt_size_abalone = len(abalone_tgt_df_y)

print("Target Set: ", abalone_tgt_df.shape)
print("Source Set: ", abalone_source_df.shape)
print("Test Set: ", abalone_test_df.shape)

print("---------------------------")

###############################################################################################################################################

Target Set:  (1585, 8)
Source Set:  (1351, 8)
Test Set:  (1241, 8)
---------------------------


In [11]:
################################## Finding best instances from the source dataset (Abalone) ######################################################

abalone_source_df["ManDis"] = ""

abalone_tgt_df_mean = []
prow = abalone_tgt_df.mean()
abalone_tgt_df_mean = [prow.Sex, prow.Length, prow.Diameter, prow.Height, prow.Shucked_weight,
       prow.Viscera_weight, prow.Shell_weight, prow.Rings]

rowidx = 0

for row in abalone_source_df.itertuples():
    row_list =[row.Sex, row.Length, row.Diameter, row.Height, row.Shucked_weight,
       row.Viscera_weight, row.Shell_weight, row.Rings]

    man_dis = 0
    for i in range(0, len(row_list)):
        tempval = abalone_tgt_df_mean[i] - row_list[i]
        man_dis = man_dis + abs(tempval)

    abalone_source_df.loc[rowidx,"ManDis"] = man_dis
    rowidx = rowidx + 1

abalone_source_df = abalone_source_df.sort_values(by =['ManDis'])
abalone_source_df = abalone_source_df.head(600) 
abalone_source_df = abalone_source_df.drop(['ManDis'], axis =1)
abalone_source_df = abalone_source_df.reset_index(drop=True)

#################### Splitting with small target set and large source and test set #############
print("Target Set: ", abalone_tgt_df.shape)
print("Source Set: ", abalone_source_df.shape)
print("Test Set: ", abalone_test_df.shape)


target_column_abalone = ['Rings']

abalone_tgt_df_y = abalone_tgt_df[target_column_abalone]
abalone_tgt_df_X = abalone_tgt_df.drop(target_column_abalone, axis = 1)

abalone_source_df_y = abalone_source_df[target_column_abalone]
abalone_source_df_X = abalone_source_df.drop(target_column_abalone, axis = 1)

abalone_test_df_y = abalone_test_df[target_column_abalone]
abalone_test_df_X = abalone_test_df.drop(target_column_abalone, axis = 1)

############## Merging the datasets ##########################################
abalone_X_df = pd.concat([abalone_tgt_df_X, abalone_source_df_X], ignore_index=True)
abalone_y_df = pd.concat([abalone_tgt_df_y, abalone_source_df_y], ignore_index=True)

abalone_np_train_X = abalone_X_df.to_numpy()
abalone_np_train_y = abalone_y_df.to_numpy()

abalone_np_test_X = abalone_test_df_X.to_numpy()
abalone_np_test_y = abalone_test_df_y.to_numpy()

abalone_np_train_y_list = abalone_np_train_y.ravel()
abalone_np_test_y_list = abalone_np_test_y.ravel()

src_size_abalone = len(abalone_source_df_y)
tgt_size_abalone = len(abalone_tgt_df_y)

print("---------------------------")

###############################################################################################################################################

Target Set:  (1361, 8)
Source Set:  (600, 8)
Test Set:  (1471, 8)
---------------------------


In [21]:
#################################### STrAdaBoost.R2 Abalone ################################################################
from two_TrAdaBoostR2 import TwoStageTrAdaBoostR2

sample_size = [len(abalone_tgt_df_X), len(abalone_source_df_X)]
n_estimators = 100
steps = 30
fold = 10
random_state = np.random.RandomState(1)

r2scorelist_stradaboost_abalone = []
rmselist_stradaboost_abalone = []

print("STrAdaboost.R2 (M + H, L)")
print("-------------------------------------------")


for x in range(0, 10):

    model_stradaboost_abalone = TwoStageTrAdaBoostR2(DecisionTreeRegressor(max_depth=6),
                        n_estimators = n_estimators, sample_size = sample_size,
                        steps = steps, fold = fold, random_state = random_state)


    model_stradaboost_abalone.fit(abalone_np_train_X, abalone_np_train_y_list)
    y_pred_stradaboost_abalone = model_stradaboost_abalone.predict(abalone_np_test_X)

    mse_stradaboost_abalone = sqrt(mean_squared_error(abalone_np_test_y, y_pred_stradaboost_abalone))
    rmselist_stradaboost_abalone.append(mse_stradaboost_abalone)
        
    r2_score_stradaboost_abalone = pearsonr(abalone_np_test_y_list, y_pred_stradaboost_abalone)
    r2_score_stradaboost_abalone = (r2_score_stradaboost_abalone[0])**2
    r2scorelist_stradaboost_abalone.append(r2_score_stradaboost_abalone)


print("RMSE of STrAdaboostR2:", statistics.mean(rmselist_stradaboost_abalone))
print("R^2 of STrAdaboostR2:", statistics.mean(r2scorelist_stradaboost_abalone))
print("\n")
print("RMSE of STrAdaboostR2:", rmselist_stradaboost_abalone)
print("R^2 of STrAdaboostR2:", r2scorelist_stradaboost_abalone)

STrAdaboost.R2 (M + H, L)
-------------------------------------------
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
RMSE of STrAdaboostR2: 7.104359518750208
R^2 of STrAdaboostR2: 0.07273975094449348


RMSE of STrAdaboostR2: [7.187388517945626, 6.666103278905509, 6.983727759335352, 7.092190448572101, 7.246114098172859, 7.194199071061588, 7.26562941657108, 7.13307493280912, 7.171607006213845, 7.103560657914999]
R^2 of STrAdaboostR2: [0.07697173632211339, 0.05698207997635008, 0.08065165039218536, 0.07198835372742526, 0.06187330358909322, 0.06925696278790539, 0.05693035129910831, 0.08724334376004313, 0.06396992847296112, 0.10152979911774947]


In [12]:
############# Transfer Learning specifications #########################
src_idx_abalone = np.arange(start=0, stop=(src_size_abalone - 1), step=1)
tgt_idx_abalone = np.arange(start=src_size_abalone, stop=((src_size_abalone + tgt_size_abalone)-1), step=1)

def get_estimator(**kwargs):
    return DecisionTreeRegressor(max_depth = 6)

kwargs_TwoTrAda = {'steps': 30,
                    'fold': 10,
                  'learning_rate': 0.1}

In [5]:
########################### TwoStagetrAdaBoostR2 Abalone #######################################

from adapt.instance_based import TrAdaBoost, TrAdaBoostR2, TwoStageTrAdaBoostR2

print("Two-Stage TrAdaboost.R2 (M + H, L)")
print("-------------------------------------------")

r2scorelist_TwoTrAda_abalone = []
rmselist_TwoTrAda_abalone = []

for x in range(0, 15):

    model_TwoTrAda_abalone = TwoStageTrAdaBoostR2(get_estimator = get_estimator, n_estimators = 1000, cv=10) #, kwargs_TwoTrAda)
    model_TwoTrAda_abalone.fit(abalone_np_train_X, abalone_np_train_y_list, src_idx_abalone, tgt_idx_abalone)

    y_pred_TwoTrAda_abalone = model_TwoTrAda_abalone.predict(abalone_np_test_X)

    mse_TwoTrAda_abalone = sqrt(mean_squared_error(abalone_np_test_y, y_pred_TwoTrAda_abalone))
    rmselist_TwoTrAda_abalone.append(mse_TwoTrAda_abalone)
        
    r2_score_TwoTrAda_abalone = pearsonr(abalone_np_test_y_list, y_pred_TwoTrAda_abalone)
    r2_score_TwoTrAda_abalone = (r2_score_TwoTrAda_abalone[0])**2
    r2scorelist_TwoTrAda_abalone.append(r2_score_TwoTrAda_abalone)

print("RMSE of Two-Stage TrAdaboost.R2:", statistics.mean(rmselist_TwoTrAda_abalone))
print("R^2 of TrAdaboostR2:", statistics.mean(r2scorelist_TwoTrAda_abalone))
print("\n")
print("RMSE of Two-Stage TrAdaboost.R2:", rmselist_TwoTrAda_abalone)
print("R^2 of TrAdaboostR2:", r2scorelist_TwoTrAda_abalone)


print("-------------------------------------------")

Two-Stage TrAdaboost.R2 (M + H, L)
-------------------------------------------
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not con

In [23]:
########################### AdaBoostR2 Transfer Learning Abalone #####################################################
from sklearn.ensemble import AdaBoostRegressor


print("Adaboost.R2 Transfer Learning (L + M, H)")
print("-------------------------------------------")

r2scorelist_AdaTr_abalone = []
rmselist_AdaTr_abalone = []

for x in range(0, 10):

    model_AdaTr_abalone = AdaBoostRegressor(DecisionTreeRegressor(max_depth = 6), learning_rate = 0.1, n_estimators = 100) 
    model_AdaTr_abalone.fit(abalone_np_train_X, abalone_np_train_y_list)

    y_pred_AdaTr_abalone = model_AdaTr_abalone.predict(abalone_np_test_X)

    mse_AdaTr_abalone = sqrt(mean_squared_error(abalone_np_test_y, y_pred_AdaTr_abalone))
    rmselist_AdaTr_abalone.append(mse_AdaTr_abalone)
        
    r2_score_AdaTr_abalone = pearsonr(abalone_np_test_y_list, y_pred_AdaTr_abalone)
    r2_score_AdaTr_abalone = (r2_score_AdaTr_abalone[0])**2
    r2scorelist_AdaTr_abalone.append(r2_score_AdaTr_abalone)

print("RMSE of Adaboost.R2:", statistics.mean(rmselist_AdaTr_abalone))
print("R^2 of AdaboostR2:", statistics.mean(r2scorelist_AdaTr_abalone ))
print("\n")
print("RMSE of Adaboost.R2:", rmselist_AdaTr_abalone)
print("R^2 of AdaboostR2:", r2scorelist_AdaTr_abalone)


print("-------------------------------------------")

Adaboost.R2 Transfer Learning (L + M, H)
-------------------------------------------
RMSE of Adaboost.R2: 4.042890482680771
R^2 of AdaboostR2: 0.010789207720236213


RMSE of Adaboost.R2: [4.084390878081856, 4.1016915315432065, 4.039704626543275, 3.8627123431635533, 3.978265863845126, 4.088636003793865, 4.021746471797686, 4.018137064753472, 4.177411653431399, 4.056208389854268]
R^2 of AdaboostR2: [0.004629193721794503, 0.003755422122530095, 0.010522141191700179, 0.016194811095439864, 0.02628247111050373, 0.004902019165764419, 0.014564707287218943, 0.012213489144649228, 0.00013458284909990042, 0.014693239513661277]
-------------------------------------------


In [24]:
########################### AdaBoostR2 Abalone #####################################################
from sklearn.ensemble import AdaBoostRegressor


print("Adaboost.R2 (L + M, H)")
print("-------------------------------------------")

r2scorelist_Ada_abalone = []
rmselist_Ada_abalone = []

for x in range(0, 10):

    model_Ada_abalone = AdaBoostRegressor(DecisionTreeRegressor(max_depth = 6), learning_rate = 0.1, n_estimators = 100) 
    model_Ada_abalone.fit(abalone_np_train_X, abalone_np_train_y_list)

    y_pred_ada_abalone = model_Ada_abalone.predict(abalone_np_test_X)

    mse_Ada_abalone = sqrt(mean_squared_error(abalone_np_test_y, y_pred_ada_abalone))
    rmselist_Ada_abalone.append(mse_Ada_abalone)
        
    r2_score_Ada_abalone = pearsonr(abalone_np_test_y_list, y_pred_ada_abalone)
    r2_score_Ada_abalone = (r2_score_Ada_abalone[0])**2
    r2scorelist_Ada_abalone.append(r2_score_Ada_abalone)

print("RMSE of Adaboost.R2:", statistics.mean(rmselist_Ada_abalone))
print("R^2 of AdaboostR2:", statistics.mean(r2scorelist_Ada_abalone))
print("\n")
print("RMSE of Adaboost.R2:", rmselist_Ada_abalone)
print("R^2 of AdaboostR2:", r2scorelist_Ada_abalone)

print("-------------------------------------------")

Adaboost.R2 (L + M, H)
-------------------------------------------
RMSE of Adaboost.R2: 4.038324962813185
R^2 of AdaboostR2: 0.008653238833924414


RMSE of Adaboost.R2: [3.994833978338952, 4.075123284107963, 4.010872668561137, 4.025382724853608, 4.092602710512061, 3.989405517089684, 4.122209654869254, 4.153006197280525, 3.9332699050625948, 3.986542987456071]
R^2 of AdaboostR2: [0.018851101666206554, 0.003737900977885546, 0.008261174383365794, 0.014510214223867782, 0.00557693603350759, 0.010807422161128635, 0.00963271172672432, 0.0006639307555298398, 0.011126867384265477, 0.0033641290267626147]
-------------------------------------------


In [194]:
######################### Kernel Mean Matching Abalone #######################################
from adapt.instance_based import KMM


print("Kernel Mean Matching")
print("-------------------------------------------")

r2scorelist_KMM_abalone = []
rmselist_KMM_abalone = []

for x in range(0, 50):

    model_KMM_abalone = KMM(get_estimator = get_estimator)
    model_KMM_abalone.fit(abalone_np_train_X, abalone_np_train_y_list, src_idx_abalone, tgt_idx_abalone)

    y_pred_KMM_abalone = model_KMM_abalone.predict(abalone_test_df_X) ##Using dataframe instead of the numpy matrix

    mse_KMM_abalone = sqrt(mean_squared_error(abalone_np_test_y, y_pred_KMM_abalone))
    rmselist_KMM_abalone.append(mse_KMM_abalone)
        
    r2_score_KMM_abalone = pearsonr(abalone_np_test_y_list, y_pred_KMM_abalone)
    r2_score_KMM_abalone = (r2_score_KMM_abalone[0])**2
    r2scorelist_KMM_abalone.append(r2_score_KMM_abalone)

    
print("RMSE of KMM:", statistics.mean(rmselist_KMM_abalone))
print("R^2 of TrAdaboostR2:", statistics.mean(r2scorelist_KMM_abalone))
print("\n")
print("RMSE of KMM:", rmselist_KMM_abalone)
print("R^2 of TrAdaboostR2:", r2scorelist_KMM_abalone)

print("-------------------------------------------")

Kernel Mean Matching
-------------------------------------------


StatisticsError: mean requires at least one data point

In [25]:
######################### Gradient Boosting Regression Transfer Learning Abalone #######################################
from sklearn.ensemble import GradientBoostingRegressor


print("Gradient Boosting Regression Transfer Learning (L + M, H)")
print("-------------------------------------------")

r2scorelist_GBRTL_abalone = []
rmselist_GBRTL_abalone = []

for x in range(0, 10):

    model_GBRTL_abalone = GradientBoostingRegressor(learning_rate = 0.1, max_depth = 6, n_estimators = 100, subsample=0.5)
    model_GBRTL_abalone.fit(abalone_np_train_X, abalone_np_train_y_list)

    y_pred_GBRTL_abalone = model_GBRTL_abalone.predict(abalone_test_df_X) ##Using dataframe instead of the numpy matrix

    mse_GBRTL_abalone = sqrt(mean_squared_error(abalone_np_test_y, y_pred_GBRTL_abalone))
    rmselist_GBRTL_abalone.append(mse_GBRTL_abalone)
        
    r2_score_GBRTL_abalone = pearsonr(abalone_np_test_y_list, y_pred_GBRTL_abalone)
    r2_score_GBRTL_abalone = (r2_score_GBRTL_abalone[0])**2
    r2scorelist_GBRTL_abalone.append(r2_score_GBRTL_abalone)


print("RMSE of GBRTL:", statistics.mean(rmselist_GBRTL_abalone))
print("R^2 of GBRTL:", statistics.mean(r2scorelist_GBRTL_abalone))
print("\n")
print("RMSE of GBRTL:", rmselist_GBRTL_abalone)
print("R^2 of GBRTL:", r2scorelist_GBRTL_abalone)


print("-------------------------------------------")

Gradient Boosting Regression Transfer Learning (L + M, H)
-------------------------------------------
RMSE of GBRTL: 3.4718335647731897
R^2 of GBRTL: 0.02821969094449157


RMSE of GBRTL: [3.3432113026183914, 3.4773643073253675, 3.4182718921683715, 3.3105328261311118, 3.8379465260522814, 3.4197886070958234, 3.4646013396578166, 3.607545524312897, 3.547168230200105, 3.2919050921697335]
R^2 of GBRTL: [0.03623650573269374, 0.02324718748267828, 0.020959209744500825, 0.026130666160258616, 0.0047736433659274985, 0.026194227287646872, 0.04889028326837783, 0.011456458580578598, 0.032535422655029164, 0.05177330516722428]
-------------------------------------------


In [26]:
######################### Gradient Boosting Regression Abalone #######################################
from sklearn.ensemble import GradientBoostingRegressor


print("Gradient Boosting Regression (L + M, H)")
print("-------------------------------------------")

r2scorelist_GBR_abalone = []
rmselist_GBR_abalone = []

for x in range(0, 10):

    model_GBR_abalone = GradientBoostingRegressor(learning_rate = 0.1, max_depth = 6, n_estimators = 100, subsample=0.5)
    model_GBR_abalone.fit(abalone_tgt_df_X, abalone_tgt_df_y)

    y_pred_GBR_abalone = model_GBR_abalone.predict(abalone_test_df_X) ##Using dataframe instead of the numpy matrix

    mse_GBR_abalone = sqrt(mean_squared_error(abalone_np_test_y, y_pred_GBR_abalone))
    rmselist_GBR_abalone.append(mse_GBR_abalone)
        
    r2_score_GBR_abalone = pearsonr(abalone_np_test_y_list, y_pred_GBR_abalone)
    r2_score_GBR_abalone = (r2_score_GBR_abalone[0])**2
    r2scorelist_GBR_abalone.append(r2_score_GBR_abalone)

print("RMSE of GBR:", statistics.mean(rmselist_GBR_abalone))
print("R^2 of GBR:", statistics.mean(r2scorelist_GBR_abalone))
print("\n")
print("RMSE of GBR:", rmselist_GBR_abalone)
print("R^2 of GBR:", r2scorelist_GBR_abalone)


print("-------------------------------------------")

Gradient Boosting Regression (L + M, H)
-------------------------------------------


/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exam

RMSE of GBR: 3.6383843342351487
R^2 of GBR: 0.09954019791546942


RMSE of GBR: [3.276675288630834, 3.361077312791927, 3.529108762647334, 3.3034151993967704, 3.421567008763177, 3.738411453227392, 4.340992520742204, 3.6374472831830014, 3.142725669175493, 4.6324228437933535]
R^2 of GBR: [0.0969200551416905, 0.11716137454619567, 0.0983236873031202, 0.08270968182463283, 0.10511820150920499, 0.10988896273274129, 0.09818595047459719, 0.10608666208264597, 0.08011486490173096, 0.10089253863813452]
-------------------------------------------


In [197]:
############################################ Transformed TrAdaBoost.R2 Abalone ########################################################
from adapt.instance_based import TrAdaBoost, TrAdaBoostR2, TwoStageTrAdaBoostR2


weights_KMM = model_KMM_abalone.weights_

val_mean = np.mean(weights_KMM)
weights_KMM = np.append(weights_KMM, val_mean)

abalone_source_df_X_trans = abalone_source_df_X.apply(lambda abalone_source_df_X: abalone_source_df_X * weights_KMM)
#concrete_source_df_X.multiply(weights_KMM, axis=0)

# concrete_np_train_X_trans = concrete_source_df_X_trans.to_numpy()

abalone_X_df = pd.concat([abalone_tgt_df_X, abalone_source_df_X_trans], ignore_index=True)
abalone_y_df = pd.concat([abalone_tgt_df_y, abalone_source_df_y], ignore_index=True)

abalone_np_train_X = abalone_X_df.to_numpy()
abalone_np_train_y = abalone_y_df.to_numpy()

abalone_np_test_X = abalone_test_df_X.to_numpy()
abalone_np_test_y = abalone_test_df_y.to_numpy()

abalone_np_train_y_list = abalone_np_train_y.ravel()
abalone_np_test_y_list = abalone_np_test_y.ravel()

# print(concrete_np_train_y_list.shape, concrete_np_train_X_trans.shape)

##################################################################
print("Transformed Two-Stage TrAdaboost.R2")
print("-------------------------------------------")

r2scorelist_TwoTrAda_abalone = []
rmselist_TwoTrAda_abalone = []

for x in range(0, 10):

    model_TwoTrAda_abalone = TwoStageTrAdaBoostR2(get_estimator = get_estimator, n_estimators = 100, cv=10) #, kwargs_TwoTrAda)
    model_TwoTrAda_abalone.fit(abalone_np_train_X, abalone_np_train_y_list, src_idx_abalone, tgt_idx_abalone)

    y_pred_TwoTrAda_abalone = model_TwoTrAda_abalone.predict(abalone_np_test_X)

    mse_TwoTrAda_abalone = sqrt(mean_squared_error(abalone_np_test_y, y_pred_TwoTrAda_abalone))
    rmselist_TwoTrAda_abalone.append(mse_TwoTrAda_abalone)
        
    r2_score_TwoTrAda_abalone = pearsonr(abalone_np_test_y_list, y_pred_TwoTrAda_abalone)
    r2_score_TwoTrAda_abalone = (r2_score_TwoTrAda_abalone[0])**2
    r2scorelist_TwoTrAda_abalone.append(r2_score_TwoTrAda_abalone)


print("RMSE of Two-Stage TrAdaboost.R2:", statistics.mean(rmselist_TwoTrAda_abalone))
print("R^2 of TrAdaboostR2:", statistics.mean(r2scorelist_TwoTrAda_abalone))
print("\n")
print("RMSE of Two-Stage TrAdaboost.R2:", rmselist_TwoTrAda_abalone)
print("R^2 of TrAdaboostR2:", r2scorelist_TwoTrAda_abalone)


print("-------------------------------------------")

Transformed Two-Stage TrAdaboost.R2
-------------------------------------------
RMSE of Two-Stage TrAdaboost.R2: 4.138833520441751
R^2 of TrAdaboostR2: 0.0182268953114228


RMSE of Two-Stage TrAdaboost.R2: [5.678807958267232, 3.9267732344829804, 3.8862125047866276, 4.234017515985184, 4.140431209808338, 4.103640520347608, 4.092348497736601, 3.552947291867083, 3.7278255366868063, 4.045330934449049]
R^2 of TrAdaboostR2: [0.08998222813259087, 0.023669815573501122, 0.02654264118817837, 0.005541930754263786, 0.005603093075105872, 0.005711250570588052, 0.001110517449232109, 0.02091489873433919, 0.0004277972727651144, 0.002764780363663507]
-------------------------------------------


In [20]:
#################################### STrAdaBoost.R2 Abalone ################################################################
from two_TrAdaBoostR2 import TwoStageTrAdaBoostR2

sample_size = [len(abalone_tgt_df_X), len(abalone_source_df_X)]
n_estimators = 100
steps = 30
fold = 10
random_state = np.random.RandomState(1)

r2scorelist_stradaboost_abalone = []
rmselist_stradaboost_abalone = []

for x in range(0, 50):

    model_stradaboost_abalone = TwoStageTrAdaBoostR2(DecisionTreeRegressor(max_depth=6),
                        n_estimators = n_estimators, sample_size = sample_size,
                        steps = steps, fold = fold, random_state = random_state)


    model_stradaboost_abalone.fit(abalone_np_train_X, abalone_np_train_y_list)
    y_pred_stradaboost_abalone = model_stradaboost_abalone.predict(abalone_np_test_X)

    mse_stradaboost_abalone = sqrt(mean_squared_error(abalone_np_test_y, y_pred_stradaboost_abalone))
    rmselist_stradaboost_abalone.append(mse_stradaboost_abalone)
        
    r2_score_stradaboost_abalone = pearsonr(abalone_np_test_y_list, y_pred_stradaboost_abalone)
    r2_score_stradaboost_abalone = (r2_score_stradaboost_abalone[0])**2
    r2scorelist_stradaboost_abalone.append(r2_score_stradaboost_abalone)


print("RMSE of STrAdaboostR2:", statistics.mean(rmselist_stradaboost_abalone))
print("R^2 of STrAdaboostR2:", statistics.mean(r2scorelist_stradaboost_abalone))
print("\n")
print("RMSE of STrAdaboostR2:", rmselist_stradaboost_abalone)
print("R^2 of STrAdaboostR2:", r2scorelist_stradaboost_abalone)

Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2


KeyboardInterrupt: 

In [47]:
################################################## Kinematics ######################################################################
#### range: [0.04 - 1.45]
#### Mid of correlation variable: theta7
#### [0, 0.6] [0.6, 0.85], [0.6, 0.85]
####################################################################################################################################
target_var_Kinematics = ['y']
colnames_Kinematics = ['theta1', 'theta2', 'theta3', 'theta4', 'theta5', 'theta6', 'theta7', 'theta8', 'y']
KinematicsData_df = pd.read_csv('UCI_regression/Kinematics/kin8nm.data', header = None,  names = colnames_Kinematics)
print("Kinematics Data")
print(KinematicsData_df.shape)

########## Corr Kinematics ################
# print("The correlation matrix is: ")
# print(KinematicsData_df.corr().sort_values('y')['y'])

##################### Splitting in 3 equal parts #######################################
# print(KinematicsData_df.sort_values('theta7')['theta7'])

drop_col_kinematics = ['theta7']

kinematics_tgt_df = KinematicsData_df.loc[(KinematicsData_df['theta7'] <= -0.5)]
kinematics_tgt_df = kinematics_tgt_df.drop(drop_col_kinematics, axis = 1)
kinematics_tgt_df = kinematics_tgt_df.reset_index(drop=True)
print("Target Set: ",kinematics_tgt_df.shape)


kinematics_source_df = KinematicsData_df.loc[(KinematicsData_df['theta7'] > -0.5) & (KinematicsData_df['theta7'] <= 0.5)]
kinematics_source_df = kinematics_source_df.drop(drop_col_kinematics, axis = 1)
kinematics_source_df = kinematics_source_df.reset_index(drop=True)
print("Source Set: ",kinematics_source_df.shape)


kinematics_test_df = KinematicsData_df.loc[(KinematicsData_df['theta7'] > 0.5)]
kinematics_test_df = kinematics_test_df.drop(drop_col_kinematics, axis = 1)
kinematics_test_df = kinematics_test_df.reset_index(drop=True)
print("Test Set: ",kinematics_test_df.shape)


# #################### Splitting with small target set and large source and test set #############
# KinematicsData_df_remain, KinematicsData_df_tgt = train_test_split(KinematicsData_df, test_size = 0.05) ## test_size = tgt size
# KinematicsData_df_source, KinematicsData_df_test = train_test_split(KinematicsData_df_remain, test_size = 0.3) ## test_size = tgt size
# print(KinematicsData_df_tgt.shape, KinematicsData_df_source.shape, KinematicsData_df_test.shape)

#################### Splitting into features and target ####################
target_column_kinematics = ['y']

kinematics_tgt_df_y = kinematics_tgt_df[target_column_kinematics]
kinematics_tgt_df_X = kinematics_tgt_df.drop(target_column_kinematics, axis = 1)

kinematics_source_df_y = kinematics_source_df[target_column_kinematics]
kinematics_source_df_X = kinematics_source_df.drop(target_column_kinematics, axis = 1)

kinematics_test_df_y = kinematics_test_df[target_column_kinematics]
kinematics_test_df_X = kinematics_test_df.drop(target_column_kinematics, axis = 1)

############## Merging the datasets ##########################################
kinematics_X_df = pd.concat([kinematics_tgt_df_X, kinematics_source_df_X], ignore_index=True)
kinematics_y_df = pd.concat([kinematics_tgt_df_y, kinematics_source_df_y], ignore_index=True)

kinematics_np_train_X = kinematics_X_df.to_numpy()
kinematics_np_train_y = kinematics_y_df.to_numpy()

kinematics_np_test_X = kinematics_test_df_X.to_numpy()
kinematics_np_test_y = kinematics_test_df_y.to_numpy()

kinematics_np_train_y_list = kinematics_np_train_y.ravel()
kinematics_np_test_y_list = kinematics_np_test_y.ravel()

src_size_kinematics = len(kinematics_source_df_y)
tgt_size_kinematics = len(kinematics_tgt_df_y)

print("---------------------------")

###############################################################################################################################################

Kinematics Data
(8192, 9)
Target Set:  (2755, 8)
Source Set:  (2598, 8)
Test Set:  (2839, 8)
---------------------------


In [28]:
######################################## Phase 1: Seeding Technique (Kinematics) ###################################################
kmeans = KMeans(n_clusters = 200, random_state=0).fit(kinematics_tgt_df)

kinematics_alternate_df = kinematics_tgt_df.copy()
kinematics_alternate_df_np = kinematics_tgt_df.to_numpy()

idxlist = []
kinematics_new_df_list = []

for rowkm in kmeans.cluster_centers_:
    mindist = -99
    rowidx = 0
    idx = 0
    for row in kinematics_alternate_df_np:
        dst = distance.euclidean(row, rowkm)

        if(dst >= mindist):
            mindist = dst
            rowidx = idx
            rowval = row

        idx = idx + 1

#     print("Row selected: ", rowidx) #Alternate_df.loc[rowidx,:]\
#     print("Min. distance: ", mindist)
#     print("Matrix shape: ", kinematics_alternate_df_np.shape)
    kinematics_new_df_list.append(rowval)
    kinematics_alternate_df = np.delete(kinematics_alternate_df_np, rowidx, 0)
    idxlist.append(rowidx)


kinematics_new_df = pd.DataFrame(np.vstack(kinematics_new_df_list))

print("Shape of dataset extracted: ")
print(kinematics_new_df.shape)
print("----------------------------------------------")

Shape of dataset extracted: 
(200, 8)
----------------------------------------------


In [29]:
##################################################### Phase 2: Seeding (Kinematics) ################################################

kinematics_alternate_source_df = kinematics_source_df[1:].copy()
kinematics_alternate_source_df_np = kinematics_alternate_source_df.to_numpy()

idxlist2 = []
kinematics_final_df_list = []

for row_nm in kinematics_new_df_list:
    min_dist = -99
    row_idx = 0
    idx_val = 0
    for row_alt in kinematics_alternate_source_df_np:
        dst = distance.euclidean(row_alt, row_nm)
        if(dst >= mindist):
            min_dist = dst
            row_idx = idx_val
            row_val = row_alt

        idx_val = idx_val + 1

#     print("Row selected: ", row_idx) #Alternate_df.loc[rowidx,:]\
#     print("Min. distance: ", min_dist)
#     print("Matrix shape: ", Elevators_alternate_source_df_np.shape)
    kinematics_final_df_list.append(row_val)
    kinematics_alternate_source_df_np = np.delete(kinematics_alternate_source_df_np, row_idx, 0)
    idxlist2.append(row_idx)


kinematics_final_df = pd.DataFrame(np.vstack(kinematics_final_df_list), columns= kinematics_source_df.columns)

print("----------------------------------------------")
print("Shape of source before :",kinematics_source_df.shape)
kinematics_source_df = pd.DataFrame(np.vstack(kinematics_alternate_source_df_np), columns= kinematics_source_df.columns)
print("Shape of source after :", kinematics_source_df.shape)

print("----------------------------------------------")
print("Shape of target before :", kinematics_tgt_df.shape)
kinematics_tgt_df = pd.concat([kinematics_tgt_df, kinematics_final_df], ignore_index=True)
print("Shape of target after :", kinematics_tgt_df.shape)

print("----------------------------------------------")

----------------------------------------------
Shape of source before : (2598, 8)
Shape of source after : (2397, 8)
----------------------------------------------
Shape of target before : (2755, 8)
Shape of target after : (2955, 8)
----------------------------------------------


In [30]:
################################## Finding best instances from the source dataset (Kinematics) ######################################################

kinematics_source_df["ManDis"] = ""

kinematics_tgt_df_mean = []
prow = kinematics_tgt_df.mean()
kinematics_tgt_df_mean = [prow.theta1, prow.theta2, prow.theta3, prow.theta4, prow.theta5, prow.theta6, prow.theta8, prow.y]

rowidx = 0

for row in kinematics_source_df.itertuples():
    row_list =[row.theta1, row.theta2, row.theta3, row.theta4, row.theta5, row.theta6, row.theta8, row.y]

    man_dis = 0
    for i in range(0, len(row_list)):
        tempval = kinematics_tgt_df_mean[i] - row_list[i]
        man_dis = man_dis + abs(tempval)

    kinematics_source_df.loc[rowidx,"ManDis"] = man_dis
    rowidx = rowidx + 1

kinematics_source_df = kinematics_source_df.sort_values(by =['ManDis'])
kinematics_source_df = kinematics_source_df.head(800) 
kinematics_source_df = kinematics_source_df.drop(['ManDis'], axis =1)
kinematics_source_df = kinematics_source_df.reset_index(drop=True)

#################### Splitting with small target set and large source and test set #############
print("Target Set: ", kinematics_tgt_df.shape)
print("Source Set: ", kinematics_source_df.shape)
print("Test Set: ", kinematics_test_df.shape)


target_column_kinematics = ['y']

kinematics_tgt_df_y = kinematics_tgt_df[target_column_kinematics]
kinematics_tgt_df_X = kinematics_tgt_df.drop(target_column_kinematics, axis = 1)

kinematics_source_df_y = kinematics_source_df[target_column_kinematics]
kinematics_source_df_X = kinematics_source_df.drop(target_column_kinematics, axis = 1)

kinematics_test_df_y = kinematics_test_df[target_column_kinematics]
kinematics_test_df_X = kinematics_test_df.drop(target_column_kinematics, axis = 1)

############## Merging the datasets ##########################################
kinematics_X_df = pd.concat([kinematics_tgt_df_X, kinematics_source_df_X], ignore_index=True)
kinematics_y_df = pd.concat([kinematics_tgt_df_y, kinematics_source_df_y], ignore_index=True)

kinematics_np_train_X = kinematics_X_df.to_numpy()
kinematics_np_train_y = kinematics_y_df.to_numpy()

kinematics_np_test_X = kinematics_test_df_X.to_numpy()
kinematics_np_test_y = kinematics_test_df_y.to_numpy()

kinematics_np_train_y_list = kinematics_np_train_y.ravel()
kinematics_np_test_y_list = kinematics_np_test_y.ravel()

src_size_kinematics = len(kinematics_source_df_y)
tgt_size_kinematics = len(kinematics_tgt_df_y)

print("---------------------------")

###############################################################################################################################################

Target Set:  (2955, 8)
Source Set:  (800, 8)
Test Set:  (2839, 8)
---------------------------


In [138]:
#################################### STrAdaBoost.R2 Active Learning Kinematics ################################################################
from two_TrAdaBoostR2 import TwoStageTrAdaBoostR2

sample_size = [len(kinematics_tgt_df_X), len(kinematics_source_df_X)]
n_estimators = 100
steps = 30
fold = 10
random_state = np.random.RandomState(1)

r2scorelist_stradaboost_kinematics = []
rmselist_stradaboost_kinematics = []

for x in range(0, 10):

    model_stradaboost_kinematics = TwoStageTrAdaBoostR2(DecisionTreeRegressor(max_depth=6),
                        n_estimators = n_estimators, sample_size = sample_size,
                        steps = steps, fold = fold, random_state = random_state)


    model_stradaboost_kinematics.fit(kinematics_np_train_X, kinematics_np_train_y_list)
    y_pred_stradaboost_kinematics = model_stradaboost_kinematics.predict(kinematics_np_test_X)


    mse_stradaboost_kinematics = sqrt(mean_squared_error(kinematics_np_test_y, y_pred_stradaboost_kinematics))

    rmselist_stradaboost_kinematics.append(mse_stradaboost_kinematics)
        
    r2_score_stradaboost_kinematics = pearsonr(kinematics_np_test_y_list, y_pred_stradaboost_kinematics)
    r2_score_stradaboost_kinematics = (r2_score_stradaboost_kinematics[0])**2

    r2scorelist_stradaboost_kinematics.append(r2_score_stradaboost_kinematics)

    
print("RMSE of STrAdaboostR2:", statistics.mean(rmselist_stradaboost_kinematics))
print("R^2 of STrAdaboostR2:", statistics.mean(r2scorelist_stradaboost_kinematics))

Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
RMSE of STrAdaboostR2: 0.2602156161812607
R^2 of STrAdaboostR2: 0.18671142089412673


In [31]:
############# Transfer Learning specifications #########################
src_idx_kinematics = np.arange(start=0, stop=(src_size_kinematics - 1), step=1)
tgt_idx_kinematics = np.arange(start=src_size_kinematics, stop=((src_size_kinematics + tgt_size_kinematics)-1), step=1)
src_idx_kinematics, tgt_idx_kinematics 

def get_estimator(**kwargs):
    return DecisionTreeRegressor(max_depth = 6)

kwargs_TwoTrAda = {'steps': 30,
                    'fold': 10,
                  'learning_rate': 0.1}

In [ ]:
########################### TwoStagetrAdaBoostR2 Kinematics #######################################

from adapt.instance_based import TrAdaBoost, TrAdaBoostR2, TwoStageTrAdaBoostR2

print("Two-Stage TrAdaboost.R2 (M + H, L)")
print("-------------------------------------------")

r2scorelist_TwoTrAda_kinematics = []
rmselist_TwoTrAda_kinematics = []

for x in range(0, 10):

    model_TwoTrAda_kinematics = TwoStageTrAdaBoostR2(get_estimator = get_estimator, n_estimators = 1000, cv=10) #, kwargs_TwoTrAda)
    model_TwoTrAda_kinematics.fit(kinematics_np_train_X, kinematics_np_train_y_list, src_idx_kinematics, tgt_idx_kinematics)

    y_pred_TwoTrAda_kinematics = model_TwoTrAda_kinematics.predict(kinematics_np_test_X)

    mse_TwoTrAda_kinematics = sqrt(mean_squared_error(kinematics_np_test_y, y_pred_TwoTrAda_kinematics))
    rmselist_TwoTrAda_kinematics.append(mse_TwoTrAda_kinematics)
        
    r2_score_TwoTrAda_kinematics = pearsonr(kinematics_np_test_y_list, y_pred_TwoTrAda_kinematics)
    r2_score_TwoTrAda_kinematics = (r2_score_TwoTrAda_kinematics[0])**2
    r2scorelist_TwoTrAda_kinematics.append(r2_score_TwoTrAda_kinematics)


print("RMSE of Two-Stage TrAdaboost.R2:", statistics.mean(rmselist_TwoTrAda_kinematics))
print("R^2 of TrAdaboostR2:", statistics.mean(r2scorelist_TwoTrAda_kinematics))
print("\n")
print("RMSE of Two-Stage TrAdaboost.R2:", rmselist_TwoTrAda_kinematics)
print("R^2 of TrAdaboostR2:", r2scorelist_TwoTrAda_kinematics)


print("-------------------------------------------")

Two-Stage TrAdaboost.R2 (M + H, L)
-------------------------------------------


In [32]:
########################### AdaBoostR2 Transfer Learning Kinematics #####################################################
from sklearn.ensemble import AdaBoostRegressor


print("Adaboost.R2 Transfer Learning (L + M, H)")
print("-------------------------------------------")

r2scorelist_AdaTL_kinematics = []
rmselist_AdaTL_kinematics = []

for x in range(0, 10):

    model_AdaTL_kinematics = AdaBoostRegressor(DecisionTreeRegressor(max_depth = 6), learning_rate = 0.1, n_estimators = 100) 
    model_AdaTL_kinematics.fit(kinematics_np_train_X, kinematics_np_train_y_list)

    y_pred_AdaTL_kinematics = model_AdaTL_kinematics.predict(kinematics_np_test_X)

    mse_AdaTL_kinematics = sqrt(mean_squared_error(kinematics_np_test_y, y_pred_AdaTL_kinematics))
    rmselist_AdaTL_kinematics.append(mse_AdaTL_kinematics)
        
    r2_score_AdaTL_kinematics = pearsonr(kinematics_np_test_y_list, y_pred_AdaTL_kinematics)
    r2_score_AdaTL_kinematics = (r2_score_AdaTL_kinematics[0])**2
    r2scorelist_AdaTL_kinematics.append(r2_score_AdaTL_kinematics)

print("RMSE of Adaboost.R2:", statistics.mean(rmselist_AdaTL_kinematics))
print("R^2 of AdaboostR2:", statistics.mean(r2scorelist_AdaTL_kinematics))
print("\n")
print("RMSE of Adaboost.R2:", rmselist_AdaTL_kinematics)
print("R^2 of AdaboostR2:", r2scorelist_AdaTL_kinematics)


print("-------------------------------------------")

Adaboost.R2 Transfer Learning (L + M, H)
-------------------------------------------
RMSE of Adaboost.R2: 0.2659715148158724
R^2 of AdaboostR2: 0.16729619376765406


RMSE of Adaboost.R2: [0.2659656037190549, 0.2649261837718794, 0.2670956547772804, 0.26571615232258594, 0.26609832438967124, 0.2665024996678089, 0.2668096115432003, 0.2653447663620251, 0.26519285038301915, 0.26606350122219846]
R^2 of AdaboostR2: [0.167621782523455, 0.17124429859452128, 0.1626685788811613, 0.1688055182381043, 0.16599725486950542, 0.16502819755139322, 0.16409103115105922, 0.1705571097177232, 0.1699819092049523, 0.16696625694466527]
-------------------------------------------


In [33]:
########################### AdaBoostR2 Kinematics #####################################################
from sklearn.ensemble import AdaBoostRegressor

print("Adaboost.R2 (L + M, H)")
print("-------------------------------------------")

r2scorelist_Ada_kinematics = []
rmselist_Ada_kinematics = []

for x in range(0, 10):

    model_Ada_kinematics = AdaBoostRegressor(DecisionTreeRegressor(max_depth = 6), learning_rate = 0.1, n_estimators = 100) 
    model_Ada_kinematics.fit(kinematics_np_train_X, kinematics_np_train_y_list)

    y_pred_Ada_kinematics = model_Ada_kinematics.predict(kinematics_np_test_X)

    mse_Ada_kinematics = sqrt(mean_squared_error(kinematics_np_test_y, y_pred_Ada_kinematics))
    rmselist_Ada_kinematics.append(mse_Ada_kinematics)
        
    r2_score_Ada_kinematics = pearsonr(kinematics_np_test_y_list, y_pred_Ada_kinematics)
    r2_score_Ada_kinematics = (r2_score_Ada_kinematics[0])**2

    r2scorelist_Ada_kinematics.append(r2_score_Ada_kinematics)


print("RMSE of Adaboost.R2:", statistics.mean(rmselist_Ada_kinematics))
print("R^2 of AdaboostR2:", statistics.mean(r2scorelist_Ada_kinematics))
print("\n")
print("RMSE of Adaboost.R2:", rmselist_Ada_kinematics)
print("R^2 of AdaboostR2:", r2scorelist_Ada_kinematics)


print("-------------------------------------------")


Adaboost.R2 (L + M, H)
-------------------------------------------
RMSE of Adaboost.R2: 0.2658658994320856
R^2 of AdaboostR2: 0.16729720695459038


RMSE of Adaboost.R2: [0.2656350317981708, 0.26580101465299133, 0.2652096234721849, 0.2660944642025111, 0.2666173194750921, 0.26598577203899565, 0.26587989648140903, 0.26588912509310586, 0.2660069141042447, 0.26553983300215034]
R^2 of AdaboostR2: [0.16791960204649392, 0.1683970291061144, 0.1702505354876456, 0.16659184736554797, 0.16342565942509318, 0.16741065769653385, 0.16832593644331073, 0.1660217341915127, 0.16581658274280411, 0.1688124850408475]
-------------------------------------------


In [ ]:
######################### Kernel Mean Matching Kinematics #######################################

from adapt.instance_based import KMM


print("Kernel Mean Matching")
print("-------------------------------------------")

r2scorelist_KMM_kinematics = []
rmselist_KMM_kinematics = []

for x in range(0, 50):

    model_KMM_kinematics = KMM(get_estimator = get_estimator)
    model_KMM_kinematics.fit(kinematics_np_train_X, kinematics_np_train_y_list, src_idx_kinematics, tgt_idx_kinematics)

    y_pred_KMM_kinematics = model_KMM_kinematics.predict(kinematics_df_test_X) ##Using dataframe instead of the numpy matrix

    mse_KMM_kinematics = sqrt(mean_squared_error(kinematics_np_test_y, y_pred_KMM_kinematics))
    rmselist_KMM_kinematics.append(mse_KMM_kinematics)
        
    r2_score_KMM_kinematics = pearsonr(kinematics_np_test_y_list, y_pred_KMM_kinematics)
    r2_score_KMM_kinematics = (r2_score_KMM_kinematics[0])**2
    r2scorelist_KMM_kinematics.append(r2_score_KMM_kinematics)


print("RMSE of KMM:", statistics.mean(rmselist_KMM_kinematics))
print("R^2 of KMM:", statistics.mean(r2scorelist_KMM_kinematics))
print("\n")
print("RMSE of KMM:", rmselist_KMM_kinematics)
print("R^2 of KMM:", r2scorelist_KMM_kinematics)


#model_KMM_concrete.weights_   #To get the weights of the algorithm

print("-------------------------------------------")

In [34]:
######################### Gradient Boosting Regression Transfer Learning Kinematics #######################################
from sklearn.ensemble import GradientBoostingRegressor


print("Gradient Boosting Regression Transfer Learning (L + M, H)")
print("-------------------------------------------")

r2scorelist_GBRTL_kinematics = []
rmselist_GBRTL_kinematics = []

for x in range(0, 10):

    model_GBRTL_kinematics = GradientBoostingRegressor(learning_rate = 0.1, max_depth = 6, n_estimators = 100, subsample=0.5)
    model_GBRTL_kinematics.fit(kinematics_np_train_X, kinematics_np_train_y_list)

    y_pred_GBRTL_kinematics = model_GBRTL_kinematics.predict(kinematics_test_df_X) ##Using dataframe instead of the numpy matrix

    mse_GBRTL_kinematics = sqrt(mean_squared_error(kinematics_np_test_y, y_pred_GBRTL_kinematics))
    rmselist_GBRTL_kinematics.append(mse_GBRTL_kinematics)
        
    r2_score_GBRTL_kinematics = pearsonr(kinematics_np_test_y_list, y_pred_GBRTL_kinematics)
    r2_score_GBRTL_kinematics = (r2_score_GBRTL_kinematics[0])**2

    r2scorelist_GBRTL_kinematics.append(r2_score_GBRTL_kinematics)


print("RMSE of GBRTL:", statistics.mean(rmselist_GBRTL_kinematics))
print("R^2 of GBRTL:", statistics.mean(r2scorelist_GBRTL_kinematics))
print("\n")
print("RMSE of GBRTL:", rmselist_GBRTL_kinematics)
print("R^2 of GBRTL:", r2scorelist_GBRTL_kinematics)


print("-------------------------------------------")

Gradient Boosting Regression Transfer Learning (L + M, H)
-------------------------------------------
RMSE of GBRTL: 0.29607404308193097
R^2 of GBRTL: 0.11351107779637477


RMSE of GBRTL: [0.2989060210564464, 0.2954357141955905, 0.29472748790034076, 0.2946587999540935, 0.29504507557417525, 0.29668810846301774, 0.2967711713729296, 0.2962477364007728, 0.29620668943672235, 0.29605362646522076]
R^2 of GBRTL: [0.10512760158825277, 0.11766685688082404, 0.11584591467497533, 0.11719211134197469, 0.11763703392080078, 0.11164896530396098, 0.11218128671586154, 0.11352638364169543, 0.11106615599898735, 0.11321846789641478]
-------------------------------------------


In [35]:
######################### Gradient Boosting Regression Kinematics #######################################
from sklearn.ensemble import GradientBoostingRegressor


print("Gradient Boosting Regression (L + M, H)")
print("-------------------------------------------")

r2scorelist_GBR_kinematics = []
rmselist_GBR_kinematics = []

for x in range(0, 10):

    model_GBR_kinematics = GradientBoostingRegressor(learning_rate = 0.1, max_depth = 6, n_estimators = 100, subsample=0.5)
    model_GBR_kinematics.fit(kinematics_tgt_df_X, kinematics_tgt_df_y)

    y_pred_GBR_kinematics = model_GBR_kinematics.predict(kinematics_test_df_X) ##Using dataframe instead of the numpy matrix

    mse_GBR_kinematics = sqrt(mean_squared_error(kinematics_np_test_y, y_pred_GBR_kinematics))
    rmselist_GBR_kinematics.append(mse_GBR_kinematics)
        
    r2_score_GBR_kinematics = pearsonr(kinematics_np_test_y_list, y_pred_GBR_kinematics)
    r2_score_GBR_kinematics = (r2_score_GBR_kinematics[0])**2
    r2scorelist_GBR_kinematics.append(r2_score_GBR_kinematics)

print("RMSE of GBR:", statistics.mean(rmselist_GBR_kinematics))
print("R^2 of GBR:", statistics.mean(r2scorelist_GBR_kinematics))
print("\n")
print("RMSE of GBR:", rmselist_GBR_kinematics)
print("R^2 of GBR:", r2scorelist_GBR_kinematics)


print("-------------------------------------------")

Gradient Boosting Regression (L + M, H)
-------------------------------------------


/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exam

RMSE of GBR: 0.3124687627207899
R^2 of GBR: 0.06646200563271637


RMSE of GBR: [0.3125261118482167, 0.3136383067399613, 0.3114797315401037, 0.3131489878216299, 0.31307053917650335, 0.3126284643382543, 0.3108825039814275, 0.31380890562776004, 0.3124107935836933, 0.31109328255034846]
R^2 of GBR: [0.06702310399855087, 0.0623595582611499, 0.06710030467751284, 0.06442314056624643, 0.06505791511559939, 0.06617524607388675, 0.07000860220228608, 0.06691022080105952, 0.06690518016741977, 0.0686567844634522]
-------------------------------------------


In [ ]:
################ Transformed TrAdaBoost.R2 Kinematics #############################################
from adapt.instance_based import TrAdaBoost, TrAdaBoostR2, TwoStageTrAdaBoostR2


weights_KMM = model_KMM_kinematics.weights_

val_mean = np.mean(weights_KMM)
weights_KMM = np.append(weights_KMM, val_mean)

kinematics_source_df_X_trans = kinematics_source_df_X.apply(lambda kinematics_source_df_X: kinematics_source_df_X * weights_KMM)
#concrete_source_df_X.multiply(weights_KMM, axis=0)

# concrete_np_train_X_trans = concrete_source_df_X_trans.to_numpy()

kinematics_X_df = pd.concat([kinematics_tgt_df_X, kinematics_source_df_X_trans], ignore_index=True)
kinematics_y_df = pd.concat([kinematics_tgt_df_y, kinematics_source_df_y], ignore_index=True)

kinematics_np_train_X = kinematics_X_df.to_numpy()
kinematics_np_train_y = kinematics_y_df.to_numpy()

kinematics_np_test_X = kinematics_test_df_X.to_numpy()
kinematics_np_test_y = kinematics_test_df_y.to_numpy()

kinematics_np_train_y_list = kinematics_np_train_y.ravel()
kinematics_np_test_y_list = kinematics_np_test_y.ravel()

# print(concrete_np_train_y_list.shape, concrete_np_train_X_trans.shape)

##################################################################
print("Transformed Two-Stage TrAdaboost.R2")
print("-------------------------------------------")

r2scorelist_TwoTrAda_kinematics = []
rmselist_TwoTrAda_kinematics = []

for x in range(0, 10):

    model_TwoTrAda_kinematics = TwoStageTrAdaBoostR2(get_estimator = get_estimator, n_estimators = 100, cv=10) #, kwargs_TwoTrAda)
    model_TwoTrAda_kinematics.fit(kinematics_np_train_X, kinematics_np_train_y_list, src_idx_kinematics, tgt_idx_kinematics)

    y_pred_TwoTrAda_kinematics = model_TwoTrAda_kinematics.predict(kinematics_np_test_X)

    mse_TwoTrAda_kinematics = sqrt(mean_squared_error(kinematics_np_test_y, y_pred_TwoTrAda_kinematics))
    rmselist_TwoTrAda_kinematics.append(mse_TwoTrAda_kinematics)
        
    r2_score_TwoTrAda_kinematics = pearsonr(kinematics_np_test_y_list, y_pred_TwoTrAda_kinematics)
    r2_score_TwoTrAda_kinematics = (r2_score_TwoTrAda_kinematics[0])**2
    r2scorelist_TwoTrAda_kinematics.append(r2_score_TwoTrAda_kinematics)


print("RMSE of Two-Stage TrAdaboost.R2:", statistics.mean(mse_TwoTrAda_kinematics))
print("R^2 of TrAdaboostR2:", statistics.mean(r2_score_TwoTrAda_kinematics))
print("\n")
print("RMSE of Two-Stage TrAdaboost.R2:", mse_TwoTrAda_kinematics)
print("R^2 of TrAdaboostR2:", r2_score_TwoTrAda_kinematics)


print("-------------------------------------------")

In [ ]:
#################################### STrAdaBoost.R2 Kinematics ################################################################
from two_TrAdaBoostR2 import TwoStageTrAdaBoostR2

sample_size = [len(kinematics_tgt_df_X), len(kinematics_source_df_X)]
n_estimators = 100
steps = 30
fold = 10
random_state = np.random.RandomState(1)

r2scorelist_stradaboost_kinematics = []
rmselist_stradaboost_kinematics = []

for x in range(0, 10):

    model_stradaboost_kinematics = TwoStageTrAdaBoostR2(DecisionTreeRegressor(max_depth=6),
                        n_estimators = n_estimators, sample_size = sample_size,
                        steps = steps, fold = fold, random_state = random_state)


    model_stradaboost_kinematics.fit(kinematics_np_train_X, kinematics_np_train_y_list)
    y_pred_stradaboost_kinematics = model_stradaboost_kinematics.predict(kinematics_np_test_X)


    mse_stradaboost_kinematics = sqrt(mean_squared_error(kinematics_np_test_y, y_pred_stradaboost_kinematics))

    rmselist_stradaboost_kinematics.append(mse_stradaboost_kinematics)
        
    r2_score_stradaboost_kinematics = pearsonr(kinematics_np_test_y_list, y_pred_stradaboost_kinematics)
    r2_score_stradaboost_kinematics = (r2_score_stradaboost_kinematics[0])**2

    r2scorelist_stradaboost_kinematics.append(r2_score_stradaboost_kinematics)

    
print("RMSE of STrAdaboostR2:", statistics.mean(rmselist_stradaboost_kinematics))
print("R^2 of STrAdaboostR2:", statistics.mean(r2scorelist_stradaboost_kinematics))

In [48]:
################################### Computer Activity ###########################################
colnames_CompAct = ['lread', 'lwrite', 'scall', 'sread', 'swrite', 'fork', 'exec', 'rchar', 'wchar', 'pgout', 'ppgout', 
                    'pgfree', 'pgscan', 'atch', 'pgin', 'ppgin', 'pflt', 'vflt', 'runqsz', 'freemem', 'freeswap', 'usr' ]
CompActData_df = pd.read_csv('UCI_regression/ComputerActivity/cpu_act.data', header = None, names = colnames_CompAct)
print("Computer Activity Data")
print(CompActData_df.shape)

########## Corr Computer Activity ################
# print("The correlation matrix is: ")
# print(CompActData_df.corr().sort_values('usr')['usr'])

##################### Splitting in 3 equal parts #######################################
# print(CompActData_df.sort_values('pgin')['pgin'])

drop_col_compact = ['pgin']

compact_tgt_df = CompActData_df.loc[(CompActData_df['pgin'] <= 1.0)]
compact_tgt_df = compact_tgt_df.drop(drop_col_compact, axis = 1)
compact_tgt_df = compact_tgt_df.reset_index(drop=True)
print("Target Set: ",compact_tgt_df.shape)


compact_source_df = CompActData_df.loc[(CompActData_df['pgin'] > 1.0) & (CompActData_df['pgin'] <= 6.0)]
compact_source_df = compact_source_df.drop(drop_col_compact, axis = 1)
compact_source_df = compact_source_df.reset_index(drop=True)
print("Source Set: ",compact_source_df.shape)


compact_test_df = CompActData_df.loc[(CompActData_df['pgin'] > 6.0)]
compact_test_df = compact_test_df.drop(drop_col_compact, axis = 1)
compact_test_df = compact_test_df.reset_index(drop=True)
print("Test Set: ",compact_test_df.shape)


#################### Splitting with small target set and large source and test set #############
# KinematicsData_df_remain, KinematicsData_df_tgt = train_test_split(KinematicsData_df, test_size = 0.05) ## test_size = tgt size
# KinematicsData_df_source, KinematicsData_df_test = train_test_split(KinematicsData_df_remain, test_size = 0.3) ## test_size = tgt size
# print(KinematicsData_df_tgt.shape, KinematicsData_df_source.shape, KinematicsData_df_test.shape)

#################### Splitting into features and target ####################
target_column_compact = ['usr']

compact_tgt_df_y = compact_tgt_df[target_column_compact]
compact_tgt_df_X = compact_tgt_df.drop(target_column_compact, axis = 1)

compact_source_df_y = compact_source_df[target_column_compact]
compact_source_df_X = compact_source_df.drop(target_column_compact, axis = 1)

compact_test_df_y = compact_test_df[target_column_compact]
compact_test_df_X = compact_test_df.drop(target_column_compact, axis = 1)

############## Merging the datasets ##########################################
compact_X_df = pd.concat([compact_tgt_df_X, compact_source_df_X], ignore_index=True)
compact_y_df = pd.concat([compact_tgt_df_y, compact_source_df_y], ignore_index=True)

compact_np_train_X = compact_X_df.to_numpy()
compact_np_train_y = compact_y_df.to_numpy()

compact_np_test_X = compact_test_df_X.to_numpy()
compact_np_test_y = compact_test_df_y.to_numpy()

compact_np_train_y_list = compact_np_train_y.ravel()
compact_np_test_y_list = compact_np_test_y.ravel()

src_size_compact = len(compact_source_df_y)
tgt_size_compact = len(compact_tgt_df_y)


print("---------------------------")

###############################################################################################################################################

Computer Activity Data
(8192, 22)
Target Set:  (2766, 21)
Source Set:  (2539, 21)
Test Set:  (2887, 21)
---------------------------


In [37]:
######################################## Phase 1: Seeding Technique (Computer Activity) ###################################################
kmeans = KMeans(n_clusters = 200, random_state=0).fit(compact_tgt_df)

compact_alternate_df = compact_tgt_df.copy()
compact_alternate_df_np = compact_tgt_df.to_numpy()

idxlist = []
compact_new_df_list = []

for rowkm in kmeans.cluster_centers_:
    mindist = -99
    rowidx = 0
    idx = 0
    for row in compact_alternate_df_np:
        dst = distance.euclidean(row, rowkm)

        if(dst >= mindist):
            mindist = dst
            rowidx = idx
            rowval = row

        idx = idx + 1

#     print("Row selected: ", rowidx) #Alternate_df.loc[rowidx,:]\
#     print("Min. distance: ", mindist)
#     print("Matrix shape: ", compact_alternate_df_np.shape)
    compact_new_df_list.append(rowval)
    compact_alternate_df = np.delete(compact_alternate_df_np, rowidx, 0)
    idxlist.append(rowidx)


compact_new_df = pd.DataFrame(np.vstack(compact_new_df_list))

print("The dataset extracted is: ")
print(compact_new_df.shape)
print("------------------------------------------------------")

##################################################### Phase 2: Seeding (Computer Activity) ################################################

compact_alternate_source_df = compact_source_df.copy()
compact_alternate_source_df_np = compact_alternate_source_df.to_numpy()

idxlist2 = []
compact_final_df_list = []

for row_nm in compact_new_df_list:
    min_dist = -99
    row_idx = 0
    idx_val = 0
    for row_alt in compact_alternate_source_df_np:
        dst = distance.euclidean(row_alt, row_nm)
        if(dst >= mindist):
            min_dist = dst
            row_idx = idx_val
            row_val = row_alt

        idx_val = idx_val + 1

#     print("Row selected: ", row_idx) #Alternate_df.loc[rowidx,:]\
#     print("Min. distance: ", min_dist)
#     print("Matrix shape: ", compact_alternate_source_df_np.shape)
    compact_final_df_list.append(row_val)
    compact_alternate_source_df_np = np.delete(compact_alternate_source_df_np, row_idx, 0)
    idxlist2.append(row_idx)


compact_final_df = pd.DataFrame(np.vstack(compact_final_df_list), columns= compact_alternate_source_df.columns)

print("----------------------------------------------")
print("Shape of source before :",compact_source_df.shape)
compact_source_df = pd.DataFrame(np.vstack(compact_alternate_source_df_np), columns= compact_source_df.columns)
print("Shape of source after :", compact_source_df.shape)

print("----------------------------------------------")
print("Shape of target before :", compact_tgt_df.shape)
Elevators_tgt_df = pd.concat([compact_tgt_df, compact_final_df], ignore_index=True)
print("Shape of target after :", compact_tgt_df.shape)

print("----------------------------------------------")

The dataset extracted is: 
(200, 21)
------------------------------------------------------
----------------------------------------------
Shape of source before : (2539, 21)
Shape of source after : (2339, 21)
----------------------------------------------
Shape of target before : (2766, 21)
Shape of target after : (2766, 21)
----------------------------------------------


In [38]:
################################## Finding best instances from the source dataset (Conputer Activity) ######################################################
compact_source_df["ManDis"] = ""

compact_tgt_df_mean = []
prow = compact_tgt_df.mean()
compact_tgt_df_mean = [prow.lread, prow.lwrite, prow.scall, prow.sread, prow.swrite, prow.fork, prow.exec, prow.rchar,
       prow.wchar, prow.pgout, prow.ppgout, prow.pgfree, prow.pgscan, prow.atch, prow.ppgin, prow.pflt, prow.vflt, prow.runqsz,
        prow.freemem, prow.freeswap, prow.usr]

rowidx = 0

for row in compact_source_df.itertuples():
    row_list =[row.lread, row.lwrite, row.scall, row.sread, row.swrite, row.fork, row.exec, row.rchar,
       row.wchar, row.pgout, row.ppgout, row.pgfree, row.pgscan, row.atch, row.ppgin, row.pflt, row.vflt, row.runqsz,
        row.freemem, row.freeswap, row.usr]

    man_dis = 0
    for i in range(0, len(row_list)):
        tempval = compact_tgt_df_mean[i] - row_list[i]
        man_dis = man_dis + abs(tempval)

    compact_source_df.loc[rowidx,"ManDis"] = man_dis
    rowidx = rowidx + 1


compact_source_df = compact_source_df.sort_values(by =['ManDis'])
compact_tgt_source_df = compact_source_df.head(650) ## For Computer Activity 650 instances
compact_source_df = compact_source_df.iloc[650:]
compact_source_df = compact_source_df.drop(['ManDis'], axis =1)
compact_tgt_source_df = compact_tgt_source_df.drop(['ManDis'], axis =1)

compact_tgt_df = pd.concat([compact_tgt_df, compact_tgt_source_df], ignore_index=True) ### This line is used only for STrAdaBoost.R2 and not for TrAdaBoost.R2
compact_source_df = compact_source_df.reset_index(drop=True)


#################### Splitting with small target set and large source and test set #############
print("Target Set: ", compact_tgt_df.shape)
print("Source Set: ", compact_source_df.shape)
print("Test Set: ", compact_test_df.shape)


compact_tgt_df_y = compact_tgt_df[target_column_compact]
compact_tgt_df_X = compact_tgt_df.drop(target_column_compact, axis = 1)

compact_source_df_y = compact_source_df[target_column_compact]
compact_source_df_X = compact_source_df.drop(target_column_compact, axis = 1)

compact_test_df_y = compact_test_df[target_column_compact]
compact_test_df_X = compact_test_df.drop(target_column_compact, axis = 1)

############## Merging the datasets ##########################################
compact_X_df = pd.concat([compact_tgt_df_X, compact_source_df_X], ignore_index=True)
compact_y_df = pd.concat([compact_tgt_df_y, compact_source_df_y], ignore_index=True)

compact_np_train_X = compact_X_df.to_numpy()
compact_np_train_y = compact_y_df.to_numpy()

compact_np_test_X = compact_test_df_X.to_numpy()
compact_np_test_y = compact_test_df_y.to_numpy()

compact_np_train_y_list = compact_np_train_y.ravel()
compact_np_test_y_list = compact_np_test_y.ravel()

src_size_compact = len(compact_source_df_y)
tgt_size_compact = len(compact_tgt_df_y)


print("---------------------------")

###############################################################################################################################################

Target Set:  (3416, 21)
Source Set:  (1689, 21)
Test Set:  (2887, 21)
---------------------------


In [7]:
#################################### STrAdaBoost.R2 compAct ################################################################
from two_TrAdaBoostR2 import TwoStageTrAdaBoostR2

sample_size = [len(compact_tgt_df_X), len(compact_source_df_X)]
n_estimators = 100
steps = 30
fold = 10
random_state = np.random.RandomState(1)

r2scorelist_stradaboost_compact = []
rmselist_stradaboost_compact = []

print("STrAdaboost.R2 (50 iter)")
print("-------------------------------------------")


for x in range(0, 50):

    model_stradaboost_compact = TwoStageTrAdaBoostR2(DecisionTreeRegressor(max_depth=6),
                        n_estimators = n_estimators, sample_size = sample_size,
                        steps = steps, fold = fold, random_state = random_state)


    model_stradaboost_compact.fit(compact_np_train_X, compact_np_train_y_list)
    y_pred_stradaboost_compact = model_stradaboost_compact.predict(compact_np_test_X)
    
    mse_stradaboost_compact = sqrt(mean_squared_error(compact_np_test_y, y_pred_stradaboost_compact))
    rmselist_stradaboost_compact.append(mse_stradaboost_compact)
        
    r2_score_stradaboost_compact = pearsonr(compact_np_test_y_list, y_pred_stradaboost_compact)
    r2_score_stradaboost_compact = (r2_score_stradaboost_compact[0])**2
    r2scorelist_stradaboost_compact.append(r2_score_stradaboost_compact)
    
    print("RMSE:", mse_stradaboost_compact)
    print("R^2:", r2_score_stradaboost_compact)

print("RMSE of STrAdaboostR2:", statistics.mean(rmselist_stradaboost_compact))
print("R^2 of STrAdaboostR2:", statistics.mean(r2scorelist_stradaboost_compact))
print("\n")
print("RMSE of STrAdaboostR2:", rmselist_stradaboost_compact)
print("R^2 of STrAdaboostR2:", r2scorelist_stradaboost_compact)

STrAdaboost.R2 (50 iter)
-------------------------------------------
Inside STrAdaBoost.R2
RMSE: 3.6518983123046462
R^2: 0.9692093146788177
Inside STrAdaBoost.R2
RMSE: 3.7263850264333964
R^2: 0.968361570455601
Inside STrAdaBoost.R2
RMSE: 3.6261029105308102
R^2: 0.9696667394709508
Inside STrAdaBoost.R2
RMSE: 3.643040399572736
R^2: 0.9691056617923778
Inside STrAdaBoost.R2
RMSE: 3.6436808494007282
R^2: 0.9692350627894134
Inside STrAdaBoost.R2
RMSE: 3.6775356230091125
R^2: 0.968589416356888
Inside STrAdaBoost.R2
RMSE: 3.7291841118516675
R^2: 0.9681432109896557
Inside STrAdaBoost.R2
RMSE: 3.719313378464132
R^2: 0.9685335100156962
Inside STrAdaBoost.R2
RMSE: 3.6643163029677566
R^2: 0.9689955667182638
Inside STrAdaBoost.R2
RMSE: 3.7763546579162486
R^2: 0.9674704912521602
Inside STrAdaBoost.R2
RMSE: 3.761955341086631
R^2: 0.9676166190945308
Inside STrAdaBoost.R2
RMSE: 3.9962167726228928
R^2: 0.963685079904088
Inside STrAdaBoost.R2
RMSE: 3.706489213431168
R^2: 0.9686063075671772
Inside STrAdaBo

In [39]:
############# Transfer Learning specifications #########################
src_idx_compact = np.arange(start=0, stop=(src_size_compact - 1), step=1)
tgt_idx_compact = np.arange(start=src_size_compact, stop=((src_size_compact + tgt_size_compact)-1), step=1)
src_idx_compact, tgt_idx_compact 

def get_estimator(**kwargs):
    return DecisionTreeRegressor(max_depth = 6)

kwargs_TwoTrAda = {'steps': 30,
                    'fold': 10,
                  'learning_rate': 0.1}

In [5]:
########################### TwoStagetrAdaBoostR2 compAct #######################################

from adapt.instance_based import TrAdaBoost, TrAdaBoostR2, TwoStageTrAdaBoostR2

print("Two-Stage TrAdaboost.R2 (L + M, H)")
print("-------------------------------------------")

r2scorelist_TwoTrAda_compact = []
rmselist_TwoTrAda_compact = []

for x in range(0, 10):

    model_TwoTrAda_compact = TwoStageTrAdaBoostR2(get_estimator = get_estimator, n_estimators = 1000, cv=10) #, kwargs_TwoTrAda)
    model_TwoTrAda_compact.fit(compact_np_train_X, compact_np_train_y_list, src_idx_compact, tgt_idx_compact)

    y_pred_TwoTrAda_compact = model_TwoTrAda_compact.predict(compact_np_test_X)

    mse_TwoTrAda_compact = sqrt(mean_squared_error(compact_np_test_y, y_pred_TwoTrAda_compact))

    rmselist_TwoTrAda_compact.append(mse_TwoTrAda_compact)
        
    r2_score_TwoTrAda_compact = pearsonr(compact_np_test_y_list, y_pred_TwoTrAda_compact)
    r2_score_TwoTrAda_compact = (r2_score_TwoTrAda_compact[0])**2

    r2scorelist_TwoTrAda_compact.append(r2_score_TwoTrAda_compact)

print("RMSE of Two-Stage TrAdaboost.R2:", statistics.mean(rmselist_TwoTrAda_compact))
print("R^2 of TrAdaboostR2:", statistics.mean(r2scorelist_TwoTrAda_compact))
print("\n")
print("RMSE of Two-Stage TrAdaboost.R2:", rmselist_TwoTrAda_compact)
print("R^2 of TrAdaboostR2:", r2scorelist_TwoTrAda_compact)


print("-------------------------------------------")

Two-Stage TrAdaboost.R2 (L + M, H)
-------------------------------------------
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not con

In [40]:
########################### AdaBoostR2 Transfer Learning compAct #####################################################
from sklearn.ensemble import AdaBoostRegressor
from keras.callbacks import EarlyStopping

#earlystopping = EarlyStopping(monitor="mean_squared_error", patience=40, verbose=1, mode='auto')
#epochs = 100, batch_size = 5, callbacks=[earlystopping])

print("Adaboost.R2 Transfer Learning (L + M, H)")
print("-------------------------------------------")

r2scorelist_AdaTL_compact = []
rmselist_AdaTL_compact = []

for x in range(0, 10):

    model_AdaTL_compact = AdaBoostRegressor(DecisionTreeRegressor(max_depth = 6), learning_rate = 0.1, n_estimators = 100) 
    model_AdaTL_compact.fit(compact_np_train_X, compact_np_train_y_list) 

    y_pred_AdaTL_compact = model_AdaTL_compact.predict(compact_np_test_X)

    mse_AdaTL_compact = sqrt(mean_squared_error(compact_np_test_y, y_pred_AdaTL_compact))
    rmselist_AdaTL_compact.append(mse_AdaTL_compact)
        
    r2_score_AdaTL_compact = pearsonr(compact_np_test_y_list, y_pred_AdaTL_compact)
    r2_score_AdaTL_compact = (r2_score_AdaTL_compact[0])**2
    r2scorelist_AdaTL_compact.append(r2_score_AdaTL_compact)


print("RMSE of Adaboost.R2:", statistics.mean(rmselist_AdaTL_compact))
print("R^2 of AdaboostR2:", statistics.mean(r2scorelist_AdaTL_compact))
print("\n")
print("RMSE of Adaboost.R2:", rmselist_AdaTL_compact)
print("R^2 of AdaboostR2:", r2scorelist_AdaTL_compact)


print("-------------------------------------------")

Adaboost.R2 Transfer Learning (L + M, H)
-------------------------------------------
RMSE of Adaboost.R2: 3.9820895289176357
R^2 of AdaboostR2: 0.9679442719271389


RMSE of Adaboost.R2: [3.978072837752613, 4.003995491203951, 4.014214047816229, 3.9572234922326195, 3.946479213097642, 3.971349246624701, 3.9920924046610264, 4.004277564301826, 3.9738504694809973, 3.9793405220047533]
R^2 of AdaboostR2: [0.968080350495381, 0.9677765049717442, 0.9676944597736903, 0.9683249512254731, 0.968613540462844, 0.9678485331157302, 0.9679100121211002, 0.9674265161444588, 0.9677812170086558, 0.9679866339523113]
-------------------------------------------


In [41]:
########################### AdaBoostR2 compAct #####################################################
from sklearn.ensemble import AdaBoostRegressor


print("Adaboost.R2 (L + M, H)")
print("-------------------------------------------")

r2scorelist_Ada_compact = []
rmselist_Ada_compact = []

for x in range(0, 10):

    model_Ada_compact = AdaBoostRegressor(DecisionTreeRegressor(max_depth = 6), learning_rate = 0.1, n_estimators = 100) 
    model_Ada_compact.fit(compact_np_train_X, compact_np_train_y_list)

    y_pred_Ada_compact = model_Ada_compact.predict(compact_np_test_X)

    mse_Ada_compact = sqrt(mean_squared_error(compact_np_test_y, y_pred_Ada_compact))
    rmselist_Ada_compact.append(mse_Ada_compact)
        
    r2_score_Ada_compact = pearsonr(compact_np_test_y_list, y_pred_Ada_compact)
    r2_score_Ada_compact = (r2_score_Ada_compact[0])**2
    r2scorelist_Ada_compact.append(r2_score_Ada_compact)


print("RMSE of Adaboost.R2:", statistics.mean(rmselist_Ada_compact))
print("R^2 of AdaboostR2:", statistics.mean(r2scorelist_Ada_compact))
print("\n")
print("RMSE of Adaboost.R2:", rmselist_Ada_compact)
print("R^2 of AdaboostR2:", r2scorelist_Ada_compact)


print("-------------------------------------------")

Adaboost.R2 (L + M, H)
-------------------------------------------
RMSE of Adaboost.R2: 3.9761828842876916
R^2 of AdaboostR2: 0.9680149045222113


RMSE of Adaboost.R2: [3.9799520055076654, 3.9698317886923786, 3.9875499743859715, 3.956380728416701, 3.979219572737315, 3.969533154791846, 3.94656096082969, 3.988936293896758, 3.977455668820057, 4.006408694798536]
R^2 of AdaboostR2: [0.9681286246087516, 0.9681278070094732, 0.9678065522857091, 0.968242820769042, 0.9679281099050986, 0.967957518629006, 0.9684606461268206, 0.9677672447138006, 0.9678986628107344, 0.9678310583636761]
-------------------------------------------


In [ ]:
######################### Kernel Mean Matching compAct #######################################
from adapt.instance_based import KMM


print("Kernel Mean Matching")
print("-------------------------------------------")

r2scorelist_KMM_compact = []
rmselist_KMM_compact = []

for x in range(0, 50):

    model_KMM_compact = KMM(get_estimator = get_estimator)
    model_KMM_compact.fit(kinematics_np_train_X, kinematics_np_train_y_list, src_idx_kinematics, tgt_idx_kinematics)

    y_pred_KMM_compact = model_KMM_compact.predict(kinematics_df_test_X) ##Using dataframe instead of the numpy matrix

    mse_KMM_compact = sqrt(mean_squared_error(compact_np_test_y, y_pred_KMM_compact))
    rmselist_Ada_compact.append(mse_KMM_compact)
        
    r2_score_KMM_compact = pearsonr(compact_np_test_y_list, y_pred_KMM_compact)
    r2_score_KMM_compact = (r2_score_KMM_compact[0])**2
    r2scorelist_Ada_compact.append(r2_score_KMM_compact)

print("RMSE of KMM:", statistics.mean(rmselist_KMM_compact))
print("R^2 of TrAdaboostR2:", statistics.mean(r2scorelist_KMM_compact))
print("\n")
print("RMSE of KMM:", rmselist_KMM_compact)
print("R^2 of TrAdaboostR2:", r2scorelist_KMM_compact)


#model_KMM_concrete.weights_   #To get the weights of the algorithm

print("-------------------------------------------")

In [42]:
######################### Gradient Boosting Regression Transfer Learning compAct #######################################
from sklearn.ensemble import GradientBoostingRegressor


print("Gradient Boosting Regression Transfer Learning (L + M, H)")
print("-------------------------------------------")

r2scorelist_GBRTL_compact = []
rmselist_GBRTL_compact = []

for x in range(0, 10):

    model_GBRTL_compact = GradientBoostingRegressor(learning_rate = 0.1, max_depth = 6, n_estimators = 100, subsample=0.5)
    model_GBRTL_compact.fit(compact_np_train_X, compact_np_train_y_list)

    y_pred_GBRTL_compact = model_GBRTL_compact.predict(compact_test_df_X) ##Using dataframe instead of the numpy matrix

    mse_GBRTL_compact = sqrt(mean_squared_error(compact_np_test_y, y_pred_GBRTL_compact))
    rmselist_GBRTL_compact.append(mse_GBRTL_compact)
        
    r2_score_GBRTL_compact = pearsonr(compact_np_test_y_list, y_pred_GBRTL_compact)
    r2_score_GBRTL_compact = (r2_score_GBRTL_compact[0])**2
    r2scorelist_GBRTL_compact.append(r2_score_GBRTL_compact)


print("RMSE of GBRTL:", statistics.mean(rmselist_GBRTL_compact))
print("R^2 of GBRTL:", statistics.mean(r2scorelist_GBRTL_compact))
print("\n")
print("RMSE of GBRTL:", rmselist_GBRTL_compact)
print("R^2 of GBRTL:", r2scorelist_GBRTL_compact)


print("-------------------------------------------")

Gradient Boosting Regression Transfer Learning (L + M, H)
-------------------------------------------
RMSE of GBRTL: 3.367750883182844
R^2 of GBRTL: 0.9743411156651872


RMSE of GBRTL: [3.3652827755790438, 3.590891846664727, 3.4165409767783337, 3.361529132105068, 3.4452831706794833, 3.2479014511351463, 3.185807879546569, 3.246993721583621, 3.3665559158017166, 3.4507219619547342]
R^2 of GBRTL: [0.9738077088761931, 0.9706665849820346, 0.9744480499196713, 0.9745558565154294, 0.9731752737402682, 0.9758906414666729, 0.9767519720138907, 0.9761415991826498, 0.9746998204058344, 0.973273649549228]
-------------------------------------------


In [43]:
######################### Gradient Boosting Regression compAct #######################################
from sklearn.ensemble import GradientBoostingRegressor


print("Gradient Boosting Regression (L + M, H)")
print("-------------------------------------------")

r2scorelist_GBR_compact = []
rmselist_GBR_compact = []

for x in range(0, 10):

    model_GBR_compact = GradientBoostingRegressor(learning_rate = 0.1, max_depth = 6, n_estimators = 100, subsample=0.5)
    model_GBR_compact.fit(compact_tgt_df_X, compact_tgt_df_y)

    y_pred_GBR_compact = model_GBR_compact.predict(compact_test_df_X) ##Using dataframe instead of the numpy matrix

    mse_GBR_compact = sqrt(mean_squared_error(compact_np_test_y, y_pred_GBR_compact))
    rmselist_GBR_compact.append(mse_GBR_compact)
        
    r2_score_GBR_compact = pearsonr(compact_np_test_y_list, y_pred_GBR_compact)
    r2_score_GBR_compact = (r2_score_GBR_compact[0])**2
    r2scorelist_GBR_compact.append(r2_score_GBR_compact)

print("RMSE of GBR:", statistics.mean(rmselist_GBR_compact))
print("R^2 of GBR:", statistics.mean(r2scorelist_GBR_compact))
print("\n")
print("RMSE of GBR:", rmselist_GBR_compact)
print("R^2 of GBR:", r2scorelist_GBR_compact)


print("-------------------------------------------")

Gradient Boosting Regression (L + M, H)
-------------------------------------------


/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exam

RMSE of GBR: 4.282758835844365
R^2 of GBR: 0.9629492694089806


RMSE of GBR: [4.282904220829364, 4.313080641992975, 4.095237330756917, 4.1661142927490555, 4.047699852334963, 4.4589879768364495, 4.453081133938799, 4.54133891134317, 4.144064988867318, 4.325079008794635]
R^2 of GBR: [0.9645400104958304, 0.9654509705269654, 0.965279164472984, 0.965984461840371, 0.9666731577366704, 0.9607285005482478, 0.9601470444885801, 0.9559179296309698, 0.9648573000600743, 0.9599141542891128]
-------------------------------------------


In [ ]:
########################################## Transformed TrAdaBoost.R2 compAct ###############################################################
from adapt.instance_based import TrAdaBoost, TrAdaBoostR2, TwoStageTrAdaBoostR2


weights_KMM = model_KMM_compact.weights_

val_mean = np.mean(weights_KMM)
weights_KMM = np.append(weights_KMM, val_mean)

compact_source_df_X_trans = compact_source_df_X.apply(lambda compact_source_df_X: compact_source_df_X * weights_KMM)
#concrete_source_df_X.multiply(weights_KMM, axis=0)
# concrete_np_train_X_trans = concrete_source_df_X_trans.to_numpy()

compact_X_df = pd.concat([compact_tgt_df_X, compact_source_df_X_trans], ignore_index=True)
compact_y_df = pd.concat([compact_tgt_df_y, compact_source_df_y], ignore_index=True)

compact_np_train_X = compact_X_df.to_numpy()
compact_np_train_y = compact_y_df.to_numpy()

compact_np_test_X = compact_test_df_X.to_numpy()
compact_np_test_y = compact_test_df_y.to_numpy()

compact_np_train_y_list = compact_np_train_y.ravel()
compact_np_test_y_list = compact_np_test_y.ravel()

# print(concrete_np_train_y_list.shape, concrete_np_train_X_trans.shape)

##################################################################
print("Transformed Two-Stage TrAdaboost.R2")
print("-------------------------------------------")

r2scorelist_TwoTrAda_compact = []
rmselist_TwoTrAda_compact = []

for x in range(0, 50):

    model_TwoTrAda_compact = TwoStageTrAdaBoostR2(get_estimator = get_estimator, n_estimators = 100, cv=10) #, kwargs_TwoTrAda)
    model_TwoTrAda_compact.fit(compact_np_train_X, compact_np_train_y_list, src_idx_compact, tgt_idx_compact)

    y_pred_TwoTrAda_compact = model_TwoTrAda_kinematics.predict(compact_np_test_X)

    mse_TwoTrAda_compact = sqrt(mean_squared_error(compact_np_test_y, y_pred_TwoTrAda_compact))
    rmselist_TwoTrAda_compact.append(mse_TwoTrAda_compact)
        
    r2_score_TwoTrAda_compact = pearsonr(compact_np_test_y_list, y_pred_TwoTrAda_compact)
    r2_score_TwoTrAda_compact = (r2_score_TwoTrAda_compact[0])**2
    r2scorelist_TwoTrAda_compact.append(r2_score_TwoTrAda_compact)


print("RMSE of Two-Stage TrAdaboost.R2:", statistics.mean(rmselist_TwoTrAda_compact))
print("R^2 of TrAdaboostR2:", statistics.mean(r2scorelist_TwoTrAda_compact))
print("\n")
print("RMSE of Two-Stage TrAdaboost.R2:", rmselist_TwoTrAda_compact)
print("R^2 of TrAdaboostR2:", r2scorelist_TwoTrAda_compact)


print("-------------------------------------------")

In [11]:
#################################### STrAdaBoost.R2 compAct ################################################################
from two_TrAdaBoostR2 import TwoStageTrAdaBoostR2

sample_size = [len(compact_tgt_df_X), len(compact_source_df_X)]
n_estimators = 100
steps = 30
fold = 10
random_state = np.random.RandomState(1)

r2scorelist_stradaboost_compact = []
rmselist_stradaboost_compact = []

for x in range(0, 50):

    model_stradaboost_compact = TwoStageTrAdaBoostR2(DecisionTreeRegressor(max_depth=6),
                        n_estimators = n_estimators, sample_size = sample_size,
                        steps = steps, fold = fold, random_state = random_state)


    model_stradaboost_compact.fit(compact_np_train_X, compact_np_train_y_list)
    y_pred_stradaboost_compact = model_stradaboost_compact.predict(compact_np_test_X)
    
    mse_stradaboost_compact = sqrt(mean_squared_error(compact_np_test_y, y_pred_stradaboost_compact))
    rmselist_stradaboost_compact.append(mse_stradaboost_compact)
        
    r2_score_stradaboost_compact = pearsonr(compact_np_test_y_list, y_pred_stradaboost_compact)
    r2_score_stradaboost_compact = (r2_score_stradaboost_compact[0])**2
    r2scorelist_stradaboost_compact.append(r2_score_stradaboost_compact)

print("RMSE of STrAdaboostR2:", statistics.mean(rmselist_stradaboost_compact))
print("R^2 of STrAdaboostR2:", statistics.mean(r2scorelist_stradaboost_compact))
print("\n")
print("RMSE of STrAdaboostR2:", rmselist_stradaboost_compact)
print("R^2 of STrAdaboostR2:", r2scorelist_stradaboost_compact)

Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STr